In [1]:
from qiskit import QuantumCircuit, transpile
# from qiskit.tools.jupyter import *
from qiskit.visualization import *
#from ibm_quantum_widgets import *
# from qiskit.providers.aer import QasmSimulator
import rustworkx as rx
# from qiskit_nature.problems.second_quantization.lattice import Lattice
from qiskit.circuit.library import EfficientSU2
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister, AncillaRegister
from qiskit.circuit import Parameter
from copy import deepcopy
from qiskit.quantum_info import SparsePauliOp
from math import pi
from qiskit.circuit.library import QAOAAnsatz
import numpy as np
from math import ceil
from qiskit import transpile
from time import time
from qiskit.circuit.library import RealAmplitudes,TwoLocal
import random 

import matplotlib.pyplot as plt
plt.rcParams.update({"font.size": 16})  # enlarge matplotlib fonts

In [2]:
#----------Sampler specific routines----------------
def extract_sub_dic(dic1, qbt_lst = [1,0]):
    # new_dic = {'00':0,'01':0,'10':0,'11':0}
    new_dic = {}
    new_dic = new_dic.fromkeys(['00','01','10','11'],0)  # intiializie new_dic keys to zero 
    # print (new_dic)
    for bin_str in dic1:
        total_qbts = len(bin_str)
        sub_bin_str = ''
        for i in range(total_qbts):
            if i in qbt_lst:
    
                sub_bin_str += bin_str[total_qbts-1-i]
                
        new_dic[sub_bin_str] += dic1[bin_str]    


    #-------Normalize to 8192-----------
    sum_val = sum(new_dic.values())
    for key in new_dic:
        # val = ceil(new_dic[key]*shots/sum_val*1.0)
        new_dic[key] = new_dic[key]/(sum_val*1.0)
    
    return new_dic  # contains {state:probs}


def extract_sub_dic3(dic1, qbt_lst = [2,1,0]):
    # new_dic = {'00':0,'01':0,'10':0,'11':0}
    new_dic = {}
    new_dic = new_dic.fromkeys(['000','001','010','011','100','101','110','111'],0)  # intiializie new_dic keys to zero 
    # print (new_dic)
    for bin_str in dic1:
        total_qbts = len(bin_str)
        sub_bin_str = ''
        for i in range(total_qbts):
            if i in qbt_lst:
    
                sub_bin_str += bin_str[total_qbts-1-i]
                
        new_dic[sub_bin_str] += dic1[bin_str]   
    
        #-------Normalize to 8192-----------
    sum_val = sum(new_dic.values())
    for key in new_dic:
        # val = ceil(new_dic[key]*shots/sum_val*1.0)
        new_dic[key] = new_dic[key]/(sum_val*1.0)
    
    return new_dic  # contains {state:probs}

def calc_avg_parity(dic1, total_qbts = 16,shots=10000):

    avg_val_lst= []
    for bin_str in dic1:
        one_counter = 0
        avg_val = 0
        for char in bin_str:
            if char == '1':
                one_counter += 1
            
        val = (-1)**one_counter
        avg_val += val * dic1[bin_str] 
        avg_val_lst.append(avg_val/shots)
    
    return avg_val_lst

def remove_ind_char(s,ind):
    u = ''
    ct = 0
    for char in s:
        if ct not in ind:
            u += char
        ct += 1
    return u

#---------a function that outputs for the given hamiltonian 'hamilton', a list of indices list whereto Pauli Operator 'Pawli' is applied--------#

def hamilt_2_meas_lst(hamilton,Pawli = 'Z'):   
    indices_lst = []
    for index in range(len(hamilton)):
        hamilt = hamilton.paulis.to_labels()[index]
        N = hamilton.num_qubits
        indices = []
        a_indices = []
        pauli_ct = hamilt.count(Pawli)
        if pauli_ct != 0:
            c_ind = -1
            for i in range(pauli_ct):
                indices.append(hamilt.index(Pawli,c_ind+1))
                c_ind = hamilt.index(Pawli,c_ind+1)
                a_indices.append(N-c_ind-1)
            a_indices.reverse()

        # print(indices)
        if a_indices != []:
            indices_lst.append(a_indices)

    return indices_lst

def generate_binary_key_dic(job_result,bin_len = 22):
    a = job_result.quasi_dists
    a = a[0]
    result_dic = {}

    for i in a.keys():
        stir = bin(i)
        stir = stir[2:]   # remove '0b' from the left
        # print('orig',stir)
        stir ="0"*(bin_len-len(stir))+stir  # zero-padding to the left
        result_dic[stir]= a[i]
        # print('mod stir',stir)

    return result_dic
    # print(result_dic)



In [3]:
#---------code that returns sub-circuit (instr) on the specific qubit------------

from qiskit.converters import circuit_to_dag, dag_to_circuit
from qiskit.dagcircuit.dagnode import DAGOpNode,DAGInNode
from qiskit.dagcircuit import DAGCircuit
from qiskit.circuit import Qubit, QuantumRegister

def truncate_circuit(qubits_kept,orig_ckt): # works only for linear circuits
  
  cpy_ansatz_opt = QuantumCircuit(orig_ckt.num_qubits)
  qubits = list(range(qubits_kept+1))
  
  gates = []
  for instruction in orig_ckt.data:
    
    add_gate = False

    qbt = instruction.qubits
    
    qb = orig_ckt.find_bit(qbt[0])
    # print(qb.index)
    if qb.index in qubits:
      # gates.append(instruction)
      add_gate = True

    if len(qbt)>1:
      qb = orig_ckt.find_bit(qbt[1])
      # print(qb.index)
      if qb.index not in qubits:
        # gates.append(instruction)
        add_gate = False
    

    if add_gate:
      # print(instruction)
      cpy_ansatz_opt.data.append(instruction)

  #--------remove idle qubits---------
  a_dg = circuit_to_dag(cpy_ansatz_opt)

  iw = a_dg.idle_wires()
  iw = list(iw)
  for q in iw:
      # print(q)
      if type(q) == Qubit:
          a_dg.remove_qubits(q)        
      #------append ancilla if u like-----------
      # qr1 = QuantumRegister(1)
      # #-----------------------------------------
  cpy_ansatz_opt = dag_to_circuit(a_dg)

  ansatz_opt_trunc = deepcopy(cpy_ansatz_opt)

  return ansatz_opt_trunc

def trunc_meas_lst(qubits_kept,orig_meas_lst):

  if type(qubits_kept) == int:
    qubits = list(range(qubits_kept+1))
  else:
     qubits = qubits_kept
    
  new_meas_lst = []
  for pair in orig_meas_lst:
    keep = True
    for element in pair:
      if element not in qubits:
        keep = False
        break
    if keep:
      new_meas_lst.append(pair)
   
  return new_meas_lst


def truncate_circuit2(qbt_kpt_lst,orig_ckt): # qbt_kpt_lst is a full list of qubits to be kept
  ansatz_opt2 = deepcopy(orig_ckt)

  # qbt_kpt_lst = [11,10,0,1,2,3,4,5,6,8,7,12,13,22,23,20,21]
  total_meas_lst = qbt_kpt_lst
  #------------------------------------------
  cr = ClassicalRegister(len(total_meas_lst))
  ansatz_opt2.add_register(deepcopy(cr))

  ct = 0
  for i in total_meas_lst:
      ansatz_opt2.measure(i,cr[ct])
      ct += 1

  a_dg = circuit_to_dag(ansatz_opt2)
  m_nodes = a_dg.named_nodes('measure')

  nodes_lst = []
  for i in range(len(m_nodes)):
      l = a_dg.ancestors(m_nodes[i])
      l = list(l)
      nodes_lst.extend(l)
      nodes_lst.append(m_nodes[i])
      
  circuit_opnodes = a_dg.op_nodes()

  for co in circuit_opnodes:
      if co not in nodes_lst:
          a_dg.remove_op_node(co)

  iw = a_dg.idle_wires()
  iw = list(iw)
  for q in iw:
      # print(q)
      if type(q) == Qubit:
          # print(q)
          a_dg.remove_qubits(q)        
      #------append ancilla if u like-----------
      # qr1 = QuantumRegister(1)
      # #-----------------------------------------
  ansatz_opt2 = dag_to_circuit(a_dg)
  ansatz_opt2.remove_final_measurements()
  
  return ansatz_opt2

def update_meas_off(ml,os):
    new_ml = []
    # for meas in ml:
    for qbts_tuple in ml:
        new_ml.append([i+os for i in qbts_tuple])
    return new_ml

In [4]:
#-------------Routine to genereate hamilt from Segments of meas_lst--------------
def meas2hamilt(measure_list=[[0,1]],num_qbts=2,Pawli_X=False,Pawli_Y=False,Pawli_Z=True,wt_list = [1],int_wt3 = 1):  # A function that converts qubit-measurement indices to hamiltonian terms
                                                    # num_qbts = 11
    Total = num_qbts-1
    
    single_lst = []
    ct = 0
    for ml in measure_list:
        cont = True
        for qbt in ml:
            if  qbt >= num_qbts or qbt < 0:
                cont = False
                break
        if cont:
            str1 = 'I'*(num_qbts)

            if Pawli_Z:
                str1 = str1[:Total-ml[0]]+'Z'+str1[Total-ml[0]+1:]
                # str1 = str1[:Total-ml[1]]+'Z'+str1[Total-ml[1]+1:]
                        
                int_wt = wt_list[ct]
                if len(ml) > 1:
                    for vv in ml[1:]:
                        str1 = str1[:Total-vv]+'Z'+str1[Total-vv+1:]

                if len(ml)>=3:
                    int_wt = int_wt3
                # print(int_wt)
                single_lst.append((str1,int_wt))

            str1 = 'I'*(num_qbts)
            if Pawli_X:
                str1 = str1[:Total-ml[0]]+'X'+str1[Total-ml[0]+1:]
                # str1 = str1[:Total-ml[1]]+'X'+str1[Total-ml[1]+1:]
                        
                int_wt = wt_list[ct]
                if len(ml) > 1:
                    for vv in ml[1:]:
                        str1 = str1[:Total-vv]+'X'+str1[Total-vv+1:]
                
                if len(ml)>=3:
                    int_wt = int_wt3
                single_lst.append((str1,int_wt))
            
            str1 = 'I'*(num_qbts)
            if Pawli_Y:
                str1 = str1[:Total-ml[0]]+'Y'+str1[Total-ml[0]+1:]
                # str1 = str1[:Total-ml[1]]+'Y'+str1[Total-ml[1]+1:]
                        
                int_wt = wt_list[ct]
                if len(ml) > 1:
                    for vv in ml[1:]:
                        str1 = str1[:Total-vv]+'Y'+str1[Total-vv+1:]
                if len(ml)>=3:
                    int_wt = int_wt3
                single_lst.append((str1,int_wt))
        ct += 1

    return  SparsePauliOp.from_list(single_lst)


In [5]:
#------------Linear Circuit Builder (obsolete)------------------

def add_row_0(num_qubits=127, cat_param_en =False, cat_param_first = 1.57):
    f_q = 0
    l_q = 9
    ckt = RealAmplitudes(num_qubits, entanglement =  [(i, i+1) for i in range(f_q,l_q)], \
                         reps=1,skip_unentangled_qubits= True)

    pval = [ 1.58603747,  3.15774053,  4.70920776,  3.13686483,  1.82610302,\
            3.1530038 , -1.57793262,  3.16143655,  1.58189184, -0.0200714 ,\
                -0.11842465, -0.11756243,  0.19288624,  0.19612199,  0.18949652,\
                    0.18952092,  0.04391567,  0.03037635,  2.82814601, -0.32167447]

###########################
    if cat_param_en:
        pval[0] = cat_param_first
        for i in range(1,len(pval)):
            pval[i] = 0 
###############################

    ckt = ckt.assign_parameters(pval)

    if not cat_param_en:
        ckt.ry(4.87, 9)

    
    meas_pair = [[0,1],[1,2],[1,3],[2,3],[3,4],[4,5],[4,6],[5,6],[6,7],[7,8],[7,9],[8,9]] # first l-r row

    meas_pair.extend([[8,9,10]])  # SHP

    meas_lst = update_meas_off(meas_pair, 0)  # note: 0 is f_q of ansatz_opt (first l-r row)
    return [ckt, meas_lst]

def add_row_rl(f_q,l_q,num_qubits=127,cat_param_en =False,cat_param_first = 1.57):
    if l_q - f_q == 12:
    # diff = 12
    # f_q = 9
    # l_q-1 = 9+11  #(ACTUAL LAST IS l_q+1)

        ckt = RealAmplitudes(num_qubits, entanglement =  [(i, i+1) for i in range(f_q,l_q-1)], \
                                    reps=1,skip_unentangled_qubits= True,parameter_prefix = 'c')

        pval = [ 1.58917942,  3.1526012 ,  4.71003514, -3.15982268,  1.8070086 ,
                3.18249442,  4.7009478 ,  3.12154665,  1.56418149,  3.14197175,
                4.72986305,  3.1335578 , -0.09061695, -0.08782126,  0.25355512,
                0.25769673,  0.22403388,  0.21857736, -0.10948278, -0.09434397,
                0.90707582,  0.93435427, -0.71817504, -0.69377693]
        ###########################
        if cat_param_en:
            pval[0] = cat_param_first
            for i in range(1,len(pval)):
                pval[i] = 0 
        ###############################

        ckt = ckt.assign_parameters(pval)
        ckt.ry(Parameter('b'), l_q)
        ckt.cx(l_q-1,l_q)
        ckt.ry(Parameter('c'), l_q)
        pval = [ 1.31458661, -1.57641476]

        ###########################
        if cat_param_en:
            for i in range(len(pval)):
                pval[i] = 0 
        ###############################

        ckt = ckt.assign_parameters(pval)
        
        meas_pair = [[0,1],[1,2],[1,3],[2,3],[3,4],[4,5],[4,6],[5,6],[6,7],[7,8],[8,9],[7,9],\
                     [9,10],[10,11],[9,11],[11,12]]   # 1st r-l row
        meas_lst = update_meas_off(meas_pair, f_q)
        return ckt, meas_lst

    else:
        return None

def add_row_lr(f_q,l_q,num_qubits=127,cat_param_en =False,cat_param_first = 1.57):
    # f_q = 21
    # l_q = 31
    if l_q - f_q == 10:
        ckt = RealAmplitudes(num_qubits, entanglement =  [(i, i+1) for i in range(f_q,l_q)], \
                                    reps=1,skip_unentangled_qubits= True)

        pval = [-1.57259659,  3.13915688,  1.56233635,  3.13982696,  4.71445113,
                3.14227401,  1.56136329,  3.11562646,  4.43328489,  3.2073208 ,
                1.63965065,  0.10646573,  0.14992916,  0.10225046,  0.10335872,
                0.15355364,  0.1747325 ,  0.11814015,  0.12636823,  0.14511724,
                0.10769702, 12.48872469]

        ###########################
        if cat_param_en:
            pval[0] = cat_param_first
            for i in range(1,len(pval)):
                pval[i] = 0 
        ###############################
        ckt = ckt.assign_parameters(pval)

        ckt.ry(4.7,l_q)

        meas_pair = [[0,1],[1,2],[0,2],[2,3],[2,4],[3,4],[4,5],[5,6],[5,7],[6,7],[7,8],[8,9],[8,10],[9,10]] #subsequent l-r row 
        meas_pair.extend([[9,10,11]])
        meas_lst = update_meas_off(meas_pair, f_q)
        return [ckt, meas_lst]

    else:
        return None


In [6]:
#-----------circuit snipping--------------

def snip_ckt(cut_qubit,ckt,meas_list, output_hamilt = False,weight_lst=[]):  # must contain ry,rx,CNOT or Rzz, adjust code for more diverse set of gates
    'the code was written in Fall 2024, updated in Spring 2025 (Mar 22)'

    qubits_lst = list(range(cut_qubit,ckt.num_qubits))
    offset = qubits_lst[0]
    total_qubits = len(qubits_lst)
    new_ansatz = QuantumCircuit(total_qubits)

    for instr in ckt.data:
        # print(instr)
        actual_qbts = instr.qubits

        qbt = ckt.find_bit(actual_qbts[0])
        # print(qbt.index)
        add_not_add = False
        
        if qbt.index in qubits_lst:
            ctr_tar = [qbt.index]
            add_not_add = True
            if len(actual_qbts) == 2:
                qbt = ckt.find_bit(actual_qbts[1])
                ctr_tar.append(qbt.index)
                if qbt.index not in qubits_lst:
                    add_not_add = False
        if add_not_add:
            # print(instr.operation)
            if instr.operation.name =='ry':
                angle = instr.operation.params[0]
                new_ansatz.ry(angle, qbt.index-offset)
            
            elif instr.operation.name =='rx':
                angle = instr.operation.params[0]
                new_ansatz.rx(angle, qbt.index-offset)

            elif instr.operation.name =='cx':
                new_ansatz.cx(ctr_tar[0]-offset,ctr_tar[1]-offset)

            elif instr.operation.name =='rzz':
                angle = instr.operation.params[0]
                new_ansatz.rzz(angle,ctr_tar[0]-offset,ctr_tar[1]-offset)



    new_meas_lst = []

    for lst in meas_list:
        new_lst = []
        for i in lst:
            new_lst.append(i-offset)
        new_meas_lst.append(new_lst)
    
    if not output_hamilt:
        return [new_ansatz,new_meas_lst]
    else:
        new_hamilt = meas2hamilt(new_meas_lst, num_qbts= len(qubits_lst),\
                                        Pawli_X=True,Pawli_Y=True,Pawli_Z=True,wt_list = weight_lst,int_wt3=1)
        return [new_ansatz,new_hamilt]


In [7]:
#-----------Gnereate new meas_lst (obsolete)-----------------
#------------for a more symmetrical (5 triangle by 5 triangle) Kagome lattice----------
from copy import deepcopy
meas_lst = []

for i in range(0,33,3):
    if i*1.0/30 != 1:
        meas_lst.append([i,i+1])
        j = i
    else:
        j = i-1
    meas_lst.append([j+1,j+2]) 
    meas_lst.append([j+1,j+3])
    meas_lst.append([j+2,j+3])
meas_lst.append([32,33])
meas_lst.append([33,34])
meas_lst.append([33,35])

for i in range(34,34+33,3):
    if i*1.0/64 != 1:
        meas_lst.append([i,i+1])
        j = i
    else:
        j = i-1
    meas_lst.append([j+1,j+2]) 
    meas_lst.append([j+1,j+3])
    meas_lst.append([j+2,j+3])
meas_lst.append([66,67])
meas_lst.append([67,68])
meas_lst.append([67,69])

for i in range(68,68+33,3):
    if i*1.0/98 != 1:
        meas_lst.append([i,i+1])
        j = i
    else:
        j = i-1
    meas_lst.append([j+1,j+2]) 
    meas_lst.append([j+1,j+3])
    meas_lst.append([j+2,j+3])

meas_lst_um = []

meas_lst_um.extend([[11,18],[11,19],[8,21],[8,22],[5,24],[5,25],[2,27],[2,28]])
meas_lst_um.extend([[29,37],[29,38],[26,40],[26,41],[23,43],[23,44],[20,46],[20,47]])
meas_lst_um.extend([[45,52],[45,53],[42,55],[42,56],[39,58],[39,59],[36,61],[36,62]])
meas_lst_um.extend([[63,71],[63,72],[60,74],[60,75],[57,77],[57,78],[54,80],[54,81]])
meas_lst_um.extend([[79,86],[79,87],[75,89],[75,90],[73,93],[73,92],[70,95],[70,96]])
meas_lst_um.extend([[0,31],[34,65],[68,99],[17,48],[51,82]])

orig_meas_lst = deepcopy(meas_lst)
# print(meas_lst)

In [8]:
#-------------circuit joining routines---------

def zero_last_param(ansatz_opt):
    par_lst = [ansatz_opt.parameters[-1]]
    par_dic = dict.fromkeys(par_lst,0)
    ansatz_opt = ansatz_opt.assign_parameters(par_dic)
    return ansatz_opt

def zero_first_param(ansatz_opt):
    par_lst = [ansatz_opt.parameters[0]]
    par_dic = dict.fromkeys(par_lst,0)
    ansatz_opt = ansatz_opt.assign_parameters(par_dic)
    return ansatz_opt

def combine_circuits(ansatz_opt, meas_lst, wt_lst, ansatz_opt2, meas_lst2, wt_lst2):
    ansatz_opt.compose(ansatz_opt2)
    meas_lst.extend(meas_lst2)
    wt_lst.extend(wt_lst2)
    return [ansatz_opt,meas_lst,wt_lst]


In [9]:
#---------------traingulation hamiltonian-------------------------
def triangle_hamiltonian(triangles,ansatz_opt,meas_lst,wt_list,offset=0):
    lst_triangle_hamilt = []

    new_triangles = deepcopy(triangles)
    for ml in triangles:
        wl = []

        new_observ_lst = []
        for observ in triangles[ml]:
            ind = meas_lst.index(observ)
            wl.append(wt_list[ind])
            #-------------Mar 28,2025--------------
            new_observ = [(qubit - offset) for qubit in observ]
            new_observ_lst.append(new_observ)
        
        if offset != 0:
            new_triangles[ml]=new_observ_lst
        #-----------------end ar 28--------------------
        # print(new_triangles[ml])
        lst_triangle_hamilt.append(meas2hamilt(new_triangles[ml], num_qbts=ansatz_opt.num_qubits,\
                                Pawli_X=True,Pawli_Y=True,Pawli_Z=True,wt_list = wl,int_wt3=1))

    return lst_triangle_hamilt
#-----------------------------------------------------------------------------

In [10]:
#-------------extract weight list---------------
def extract_wt_lst(seg_meas_lst,meas_lst,wt_list):
    new_wt_lst = []
    for pair in seg_meas_lst:
        new_wt_lst.append(wt_list[meas_lst.index(pair)])

    return new_wt_lst

In [11]:
#--------------ODR reference circuit: Ordinary Dimers----------------
# ansatz_opt = RealAmplitudes(total_qubits, entanglement =  [(i, i+1) for i in range(0,16)], \
#                          reps=1,skip_unentangled_qubits= True, flatten = True)
# #---------First Leg--------------------

# par_lst = [ansatz_opt.parameters[-1]]
# par_dic = dict.fromkeys(par_lst,0)
# ansatz_opt = ansatz_opt.assign_parameters(par_dic)
# vals = [pi/2,pi,-pi/2,pi,pi/2,pi,-pi/2,pi]
# vals.extend(vals)  # 8-
# vals.extend(vals)  # 16-
# vals.extend(vals)  # 32-
# # # vals.extend(vals)  # 64-

# par_lst = ansatz_opt.parameters[:16]
# par_dic = {par_lst[i]: vals[i] for i in range(len(par_lst))}
# # print(par_dic)
# ansatz_opt = ansatz_opt.assign_parameters(par_dic)

# pval = [1.57284275, 0.71449939, 0.7142817 , 0.45847433, 0.44406068,
#        0.29146049, 0.32868508, 0.60830634, 0.62345832, 0.34534752,
#        0.35037029, 0.07288814, 0.06860078, 0.11563758, 0.08872128,
#        0.39373035, 0.36660837]
# ansatz_opt = ansatz_opt.assign_parameters(pval)

# #---------Second Leg--------------------
# ansatz_opt2 = RealAmplitudes(total_qubits, entanglement =  [(i, i+1) for i in range(16,20)], \
#                          reps=1,skip_unentangled_qubits= True, flatten = True)
# par_lst = [ansatz_opt2.parameters[-1],ansatz_opt2.parameters[0]]
# par_dic = dict.fromkeys(par_lst,0)
# ansatz_opt2 = ansatz_opt2.assign_parameters(par_dic)

# ansatz_opt = ansatz_opt.compose(ansatz_opt2)
# pval = [3.13978781, 4.72023257, 3.14393196, 1.58363098, 0.17900723,
#        0.15908275, 0.22211639, 0.26288713]
# ansatz_opt = ansatz_opt.assign_parameters(pval)

# #---------Third Leg--------------------
# ansatz_opt2 = RealAmplitudes(total_qubits, entanglement =  [(i, i+1) for i in range(20,24)], \
#                          reps=1,skip_unentangled_qubits= True, flatten = True)
# par_lst = [ansatz_opt2.parameters[-1],ansatz_opt2.parameters[0]]
# par_dic = dict.fromkeys(par_lst,0)
# ansatz_opt2 = ansatz_opt2.assign_parameters(par_dic)

# ansatz_opt = ansatz_opt.compose(ansatz_opt2)
# pval = [3.13978781, 4.72023257, 3.14393196, 1.58363098, 0.17900723,
#        0.15908275, 0.22211639, 0.26288713]
# pval = pval = [ 0.60375048,  2.38914412, -0.33512348,  0.1242711 ,  3.558046  ,
#         0.1280878 ,  2.10037487, -0.83521025]
# pval = [ 0.31723848,  1.53930793, -0.72498341,  0.771435  ,  3.67818636,
#         0.75093205,  1.4962306 , -1.25938888]
# ansatz_opt = ansatz_opt.assign_parameters(pval)

#---------Fourth Leg--------------------
# ansatz_opt2 = RealAmplitudes(total_qubits, entanglement =  [(i, i+1) for i in range(24,28)], \
#                          reps=1,skip_unentangled_qubits= True, flatten = True)
# par_lst = [ansatz_opt2.parameters[-1],ansatz_opt2.parameters[0]]
# par_dic = dict.fromkeys(par_lst,0)
# ansatz_opt2 = ansatz_opt2.assign_parameters(par_dic)

# ansatz_opt = ansatz_opt.compose(ansatz_opt2)
# pval = [3.13978781, 4.72023257, 3.14393196, 1.58363098, 0.17900723,
#        0.15908275, 0.22211639, 0.26288713]

# pval = [ 0.60375048,  2.38914412, -0.33512348,  0.1242711 ,  3.558046  ,
#         0.1280878 ,  2.10037487, -0.83521025]
# ansatz_opt = ansatz_opt.assign_parameters(pval)

#---------Fifth Leg--------------------
# ansatz_opt2 = RealAmplitudes(total_qubits, entanglement =  [(i, i+1) for i in range(28,32)], \
#                          reps=1,skip_unentangled_qubits= True, flatten = True)
# par_lst = [ansatz_opt2.parameters[-1],ansatz_opt2.parameters[0]]
# par_dic = dict.fromkeys(par_lst,0)
# ansatz_opt2 = ansatz_opt2.assign_parameters(par_dic)

# ansatz_opt = ansatz_opt.compose(ansatz_opt2)
# # pval = [3.13978781, 4.72023257, 3.14393196, 1.58363098, 0.17900723,
# #        0.15908275, 0.22211639, 0.26288713]
# # ansatz_opt = ansatz_opt.assign_parameters(pva

#---------Banka------------------
total_qubits = 46
ansatz_opt = RealAmplitudes(total_qubits, entanglement =  [(i, i+1) for i in range(4)], \
                         reps=1,skip_unentangled_qubits= True, flatten = True)
par_lst = [ansatz_opt.parameters[-1]]
par_dic = dict.fromkeys(par_lst,0)
ansatz_opt = ansatz_opt.assign_parameters(par_dic)

pval = [1.57, 3.13978781, 4.72023257, 3.14393196, 1.58363098, 0.17900723,
       0.15908275, 0.22211639, 0.26288713]
ansatz_opt = ansatz_opt.assign_parameters(pval)


for j in range(4,44,4):
    ansatz_opt2 = RealAmplitudes(total_qubits, entanglement =  [(i, i+1) for i in range(j,j+4)], \
                            reps=1,skip_unentangled_qubits= True, flatten = True)
    par_lst = [ansatz_opt2.parameters[-1],ansatz_opt2.parameters[0]]
    par_dic = dict.fromkeys(par_lst,0)
    ansatz_opt2 = ansatz_opt2.assign_parameters(par_dic)

    pval = [3.13978781, 4.72023257, 3.14393196, 1.58363098, 0.17900723, 0.15908275, 0.22211639, 0.26288713]
    ansatz_opt2 = ansatz_opt2.assign_parameters(pval)

    ansatz_opt = ansatz_opt.compose(ansatz_opt2)


In [12]:
#-------------latest pval Mar 29-----------
pval_17 = [-2.69848778, -2.55356058,  1.45064112,  0.04608382,  1.63006203,
       -2.22481039,  3.80166733,  1.50300589,  3.1708756 ,  4.78240089,
        3.27625729,  1.39164337, -0.05081166,  1.55493788, -0.10796239,
        1.5811422 , -0.68432028,  2.36365045,  1.4044731 ,  2.03151187,
        3.72710211,  2.67298816, -0.46279265,  3.1040088 ,  3.10425794,
        2.96650794,  0.82040649,  0.9388199 ,  2.69065931,  2.6065425 ,
       -0.9616773 ,  2.07364382,  0.30083547,  3.54372905,  1.37070941,
        4.43504071, -1.89053501]

pval_31 = [ 1.35540254,  0.76923324,  2.31558056,  4.86299794,  0.06449849,
        4.78737204,  3.01345579,  1.51295826,  0.1493654 ,  1.37867061,
        2.8178899 ,  4.76367566,  3.02525994,  1.29723065,  2.42650267,
       -1.26662232,  2.09623564,  1.9739732 , -0.23335944,  3.02666663,
        1.76585945,  1.67692686,  2.94746454, -0.31378616,  2.3569576 ,
        2.40259852,  2.65042681,  2.7151431 , -0.78802341]

pval_47 = [ 1.62747656, -0.20477723,  1.50467251, -0.01489867,  1.43294777,
       -0.148296  ,  1.75059814,  3.18235766, -1.57680895,  3.21437597,
        1.48066687,  3.95755439,  3.78340722,  1.2959501 ,  3.3190967 ,
       -1.72997831,  3.04225718,  3.58692535,  0.42352664,  3.13781937,
        0.02097631,  3.29047858,  0.2139608 ,  5.77860673,  5.90176802,
        1.26461379,  1.05541141,  3.46048972,  3.47676703,  3.64158559,
        0.85906821,  0.91924472,  1.32535243,  1.44111167]
pval_47 = [ 4.76209836,  3.23488779,  1.35157232, -1.22610726,  6.55905339,
        4.82015527,  3.121501  ,  1.61736275,  2.87247779, -1.36563313,
        2.78057172,  1.53447474,  0.10845641,  1.6372765 ,  5.38181691,
        2.37264688,  1.35282896,  1.85306641,  1.97227951,  2.03529668,
       -0.81267964,  2.38952307,  0.92781184,  0.92375084,  3.77287841,
        3.76708863,  0.40673604,  0.41898966,  3.31405209,  0.2463061 ,
       -0.07710409,  3.01987722,  3.01027225]

pval_64 = [ 4.67396404,  1.96694093,  0.20419715,  1.56473923,  0.4815241 ,
        0.94186191,  1.2629664 ,  2.82251419,  4.73780425,  3.25079959,
        1.88277687,  6.74077426,  1.03911546,  1.91626258,  3.78136576,
        4.26241306,  1.32862485,  2.23048067,  2.31523281,  5.55952808,
        3.92239234,  0.52537007,  3.71905084,  0.64759608,  1.01048513,
        2.52908387,  2.78203761,  1.22732597, -1.98223115,  0.95465395,
        1.39361556,  1.19181678,  7.40029189] # Mar 29

pval_78 =[-1.39621951,  4.11698568,  2.35216128,  1.57248503,  3.01900197,
        4.69370813,  2.87007518,  1.69681091, -0.43463274,  1.142559  ,
        4.71351505, -0.07455838, -1.59546864,  3.01057984,  1.5685812 ,
        2.40729946,  2.3555963 ,  2.52136628,  0.46721372,  0.42991808,
        2.07435373,  2.08505705,  2.57490085, -0.56538711,  8.64884847,
        5.95678419,  2.80908647,  1.36074678,  1.33691673] # Mar 29

pval_91 = [-1.21232886,  4.20596569,  0.79903221, -1.57736928,  3.049245,
        1.5658082 , -0.03712853,  1.31003161, -0.09228905,  1.45289271,
        2.93754344,  4.42058978, 11.20001709,  1.84709024,  1.54885132,
       -1.81267134,  3.51521324,  3.49457594, -0.06041762,  3.01462468,
       -0.64701781,  2.2657089 ,  1.92398521,  1.94386415,  8.85951017]

pval_103 = [ 1.63234885, -0.09413499,  1.57308982, -0.08182821,  1.64138344,
       -0.24216009,  1.8643335 ,  2.33005285,  4.11637454,  4.46833956,
        2.63980969,  1.1299389 ,  1.98393887,  4.41047903,  1.36040219,
        2.90792109, -0.04003132,  0.11925171,  3.40362275,  3.16464766,
        3.03889047,  2.84237739,  2.63041011,  2.49549261, -0.50402723]


In [13]:
perturb = 0.0
pval_17 = [i+random.gauss(0,i*perturb) for i in pval_17]
pval_31 = [i+random.gauss(0,i*perturb) for i in pval_31]
pval_47 = [i+random.gauss(0,i*perturb) for i in pval_47]
pval_64 = [i+random.gauss(0,i*perturb) for i in pval_64]
pval_78 = [i+random.gauss(0,i*perturb) for i in pval_78]
pval_91 = [i+random.gauss(0,i*perturb) for i in pval_91]
pval_103 = [i+random.gauss(0,i*perturb) for i in pval_103]

In [14]:
#----------Set total qubits----------
total_qubits = 33
total_qubits = 49
total_qubits  = 65
total_qubits  = 79
total_qubits  = 91
total_qubits  = 103

In [15]:
#----------------first row-------------
extra_param_ct = 0

init_qbt = 0
end_qbt = 19

ent = [(i,i+1) for i in range(init_qbt,end_qbt-1)]

ansatz_opt = RealAmplitudes(total_qubits,  entanglement =  ent, reps=1, \
                      skip_final_rotation_layer=False, flatten=True, parameter_prefix='θ1',skip_unentangled_qubits=True)



meas_lst = [[2,3],[3,4],[2,4],[4,5],[5,6],[6,7],[5,7],[7,8],[1,7],[1,8],[8,9],[9,10],[8,10],[10,11],[11,12],[11,13],[12,13],[0,13],\
            [0,14],[13,14],[14,15],[14,16],[15,16],[16,17]]

meas_lst.remove([0,13])
meas_lst.remove([0,14])
meas_lst.remove([1,7])
meas_lst.remove([1,8])

opf_lst = [[0,13],[0,14],[13,14],[1,7],[1,8],[7,8]]
opf_lst = [[13,14],[7,8]]

wt_list = [1 for i in range(len(meas_lst))]

for pair in opf_lst:
    wt_list[meas_lst.index(pair)] = 1.8

ansatz_opt = zero_last_param(ansatz_opt)

ansatz_opt = ansatz_opt.assign_parameters(pval_17)

#-------------------end first row--------------------------------------
triangles17 = {234:[[2,3],[3,4],[2,4]],45:[[4,5]],567:[[5,6],[6,7],[5,7]],\
             178:[[7,8],[1,7],[1,8]],8910:[[8,9],[9,10],[8,10]],\
                1011:[[10,11]],111213:[[11,12],[11,13],[12,13]],\
                    1314:[[13,14],[0,13],[0,14]],141516:[[14,15],[14,16],[15,16]],\
                        1617:[[16,17]]}

triangles17 = {234:[[2,3],[3,4],[2,4]],45:[[4,5]],567:[[5,6],[6,7],[5,7]],\
             178:[[7,8]],8910:[[8,9],[9,10],[8,10]],\
                1011:[[10,11]],111213:[[11,12],[11,13],[12,13]],\
                    1314:[[13,14]],141516:[[14,15],[14,16],[15,16]],\
                        1617:[[16,17]]}

lst_triangle_hamilt_17 = triangle_hamiltonian(triangles17,ansatz_opt2,meas_lst,wt_list)


In [16]:
#----------------2nd row--------------------------------
init_qbt = 18
end_qbt = 32

ent = [(i,i+1) for i in range(init_qbt,end_qbt)]

ansatz_opt2 = TwoLocal(total_qubits, ['ry'], 'cx',  entanglement =  ent, reps=1, \
                      skip_final_rotation_layer=False, flatten=True, parameter_prefix='θ1',skip_unentangled_qubits=True)


meas_lst2 = [[18,19],[19,20],[19,21],[20,21],[21,22],[21,23],\
             [22,23],[23,24],[24,25],[24,26],[25,26],\
                [26,27],[27,28],[27,29],[28,29],[29,30],[30,31]]     
meas_lst2.extend([[17,18],[17,19]])
# meas_lst2.extend([[15,23],[15,24],[9,29],[9,30],[12,26],[12,27],[6,32],[6,33]])

# # opf_lst = [[15,23],[15,24],[9,29],[9,30],
opf_lst = [[23,24],[29,30]]
wt_list2 = [1 for i in range(len(meas_lst2))]

for pair in opf_lst:
    wt_list2[meas_lst2.index(pair)] = 1.84

ansatz_opt2 = zero_last_param(ansatz_opt2)

# [ansatz_opt,hamilt_single]=snip_ckt(18,ansatz_opt2,meas_lst2,True, wt_list2)

# hamilt_single = meas2hamilt(meas_lst2, num_qbts=ansatz_opt.num_qubits,\
#                             Pawli_X=True,Pawli_Y=True,Pawli_Z=True,wt_list = wt_list2,int_wt3=1)

ansatz_opt2 = ansatz_opt2.assign_parameters(pval_31)


#---important tweak-------
ansatz_opt3 = QuantumCircuit(total_qubits)
ansatz_opt3.ry(Parameter('a'),init_qbt)
ansatz_opt2 = ansatz_opt3.compose(ansatz_opt2)

#------merge-------
ansatz_opt = ansatz_opt.compose(ansatz_opt2)
meas_lst.extend(meas_lst2)
wt_list.extend(wt_list2)
hamilt_single = meas2hamilt(meas_lst, num_qbts=ansatz_opt.num_qubits,\
                            Pawli_X=True,Pawli_Y=True,Pawli_Z=True,wt_list = wt_list,int_wt3=1)

#---------------------end second row -----------------------
triangles31 = {0:[[17,18],[18,19],[17,19]],1: [[19,20],[19,21],[20,21]],2:[[21,22],[21,23],[22,23]],\
             3: [[23,24]],4: [[24,25],[24,26],[25,26]],5:[[26,27]],6: [[27,28],[27,29],[28,29]],7:[[29,30]],8:[[30,31]]}  # 18-31

lst_triangle_hamilt_31 = triangle_hamiltonian(triangles31,ansatz_opt2,meas_lst2,wt_list2)


In [17]:
#----------------3rd row--------------------
init_qbt = 32
end_qbt = 48

ent = [(i,i+1) for i in range(init_qbt,end_qbt)]

ansatz_opt2 = TwoLocal(total_qubits, ['ry'], 'cx',  entanglement =  ent, reps=1, \
                      skip_final_rotation_layer=False, flatten=True, parameter_prefix='θ1',skip_unentangled_qubits=True)

meas_lst2 = [[30,32],[31,32],[32,33],[34,35],[33,35],[35,36],[33,36],[36,37],[36,38],[37,38],[38,39],[39,40],\
             [38,40],[40,41],[41,42],[41,43],[42,43],\
            [43,44],[44,45],[44,46],[45,46],[46,47]]            

# meas_lst2.extend([[31,40],[31,41],[28,43],[28,44],[25,46],[25,47],[22,50],[22,49],[20,52],[20,53]])
# opf_lst = [[15,23],[15,24],[9,29],[9,30],

opf_lst = [[43,44],[35,36]]
wt_list2 = [1 for i in range(len(meas_lst2))]
ansatz_opt2 = zero_last_param(ansatz_opt2)


for pair in opf_lst:
    wt_list2[meas_lst2.index(pair)]=1.88

# [ansatz_opt,hamilt_single]=snip_ckt(init_qbt,ansatz_opt2,meas_lst2,True, wt_list2)

#---------combine test------------------
ansatz_opt2 = ansatz_opt2.assign_parameters(pval_47)

#---important tweak-------
ansatz_opt3 = QuantumCircuit(total_qubits)
ansatz_opt3.ry(-1.53,32)
ansatz_opt3.ry(Parameter('b'),32)
ansatz_opt2 = ansatz_opt3.compose(ansatz_opt2)

ansatz_opt = ansatz_opt.compose(ansatz_opt2)
meas_lst.extend(meas_lst2)
wt_list.extend(wt_list2)

hamilt_single = meas2hamilt(meas_lst, num_qbts=ansatz_opt.num_qubits,\
                            Pawli_X=True,Pawli_Y=True,Pawli_Z=True,wt_list = wt_list,int_wt3=1)

#------------------------------------end third row --------------------------

triangles47 = {0:[[32,33]],1:[[34,35]],2:[[35,36],[33,36],[33,35]],3:[[36,37],[36,38],[37,38]],\
                4:[[38,39],[39,40],[38,40]],5:[[40,41]],6:[[41,42],[41,43],[42,43]],\
                    7: [[43,44]],8:[[44,45],[44,46],[45,46]],9:[[46,47]]}   

lst_triangle_hamilt_47 = triangle_hamiltonian(triangles47,ansatz_opt2,meas_lst2,wt_list2)

In [18]:
#-----------------4th row--------------------------
init_qbt = 48
end_qbt = 64

ent = [(i,i+1) for i in range(init_qbt,end_qbt)]

ansatz_opt2 = TwoLocal(total_qubits, ['ry'], 'cx',  entanglement =  ent, reps=1, \
                      skip_final_rotation_layer=False, flatten=True, parameter_prefix='θ1',skip_unentangled_qubits=True)


meas_lst2 = [[48,49],[49,50],[50,51],[50,52],[51,52],[52,53],[53,54],[52,54],[54,55],\
             [53,56],[53,57],[56,57],[57,58],[57,59],[58,59],[59,60],[60,61],[59,61],[61,62],[62,63]]  # omitting 47,49
#---append [[46,47]] for optimization

ansatz_opt2 = zero_last_param(ansatz_opt2)


wt_list2 = [1 for i in range(len(meas_lst2))]

opf_lst = [[49,50],[61,62]]
for pair in opf_lst:
    wt_list2[meas_lst2.index(pair)]=1.9
    
ansatz_opt2 = ansatz_opt2.assign_parameters(pval_64)

#---important tweak-------
ansatz_opt3 = QuantumCircuit(total_qubits)
ansatz_opt3.ry(Parameter('c'),init_qbt)

ansatz_opt2 = ansatz_opt3.compose(ansatz_opt2)
ansatz_opt = ansatz_opt.compose(ansatz_opt2)

# meas_lst.extend(meas_lst2)
# wt_list.extend(wt_list2)
# ansatz_opt = ansatz_opt.compose(ansatz_opt2)


                                                            # [ansatz_opt,hamilt_single]=snip_ckt(48,ansatz_opt2,meas_lst2,True, wt_list2)


meas_lst.extend(meas_lst2)
wt_list.extend(wt_list2)

triangles64 = {0:[[48,49]],1:[[49,50]],2:[[50,51],[50,52],[51,52]],3:[[52,53],[52,54],[53,54]],4:[[54,55]],\
               5:[[53,56],[56,57],[53,57]],6:[[57,58],[57,59],[58,59]],\
                7:[[59,60],[60,61],[59,61]],8:[[61,62]],9:[[62,63]]}

lst_triangle_hamilt_64 = triangle_hamiltonian(triangles64,ansatz_opt,meas_lst2,wt_list2,48)


In [19]:
#----------------5th row----------------------------
init_qbt = 64
end_qbt = 78

ent = [(i,i+1) for i in range(init_qbt,end_qbt)]

ansatz_opt2 = TwoLocal(total_qubits, ['ry'], 'cx',  entanglement =  ent, reps=1, \
                      skip_final_rotation_layer=False, flatten=True, parameter_prefix='θ1',skip_unentangled_qubits=True)


meas_lst2 = [[64,65],[65,66],[66,67],[65,67],[67,68],[68,69],[69,70],[68,70],[70,71],[71,72],\
             [72,73],[72,74],[71,74],[74,75],[75,76],[74,76],[76,77]]  

ansatz_opt2 = zero_last_param(ansatz_opt2)

ansatz_opt2 = ansatz_opt2.assign_parameters(pval_78)

wt_list2 = [1 for i in range(len(meas_lst2))]

opf_lst = [[67,68],[71,72]]
for pair in opf_lst:
    wt_list2[meas_lst2.index(pair)]=1.81
    

# [ansatz_opt,hamilt_single]=snip_ckt(64,ansatz_opt2,meas_lst2,True, wt_list2)

triangles78 = {0:[[64,65]],1:[[65,66],[66,67],[65,67]],2: [[67,68]],3:[[68,69],[69,70],[68,70]],\
               4:[[70,71]],5:[[72,73]],6:[[71,72],[72,74],[71,74]],7:[[74,75],[75,76],[74,76]],8:[[76,77]]}

lst_triangle_hamilt_78 = triangle_hamiltonian(triangles78,ansatz_opt,meas_lst2,wt_list2,64)

#---important tweak-------
ansatz_opt3 = QuantumCircuit(total_qubits)
ansatz_opt3.ry(Parameter('d'),init_qbt)

ansatz_opt2 = ansatz_opt3.compose(ansatz_opt2)
# ansatz_opt = ansatz_opt.compose(ansatz_opt2)
# meas_lst.extend(meas_lst2)
# wt_list.extend(wt_list2)

ansatz_opt = ansatz_opt.compose(ansatz_opt2)
meas_lst.extend(meas_lst2)
wt_list.extend(wt_list2)

In [20]:
#----------------6th row----------------------------
init_qbt = 78
end_qbt = 90

ent = [(i,i+1) for i in range(init_qbt,end_qbt)]

ansatz_opt2 = TwoLocal(total_qubits, ['ry'], 'cx',  entanglement =  ent, reps=1, \
                      skip_final_rotation_layer=False, flatten=True, parameter_prefix='θ1',skip_unentangled_qubits=True)


meas_lst2 = [[76,78],[77,78],[78,79],[79,80],[79,81],[80,81],[81,82],[82,83],\
             [83,84],[82,84],[84,85],[85,86],[85,87],[86,87],[87,88],[88,89]]  

ansatz_opt2 = zero_last_param(ansatz_opt2)

ansatz_opt2 = ansatz_opt2.assign_parameters(pval_91)

wt_list2 = [1 for i in range(len(meas_lst2))]

opf_lst = [[81,82],[87,88]]
for pair in opf_lst:
    wt_list2[meas_lst2.index(pair)]=1.85
    

# [ansatz_opt,hamilt_single]=snip_ckt(77,ansatz_opt2,meas_lst2,True, wt_list2)

triangles91 = {0:[[76,78],[77,78]],1:[[78,79]],2: [[79,80],[79,81],[80,81]],3:[[81,82]],\
               4:[[82,83],[83,84],[82,84]],5:[[84,85]],6:[[85,86],[85,87],[86,87]],7:[[87,88]],8:[[88,89]]}

lst_triangle_hamilt_91 = triangle_hamiltonian(triangles91,ansatz_opt,meas_lst2,wt_list2,77)

#---important tweak-------
ansatz_opt3 = QuantumCircuit(total_qubits)
ansatz_opt3.ry(Parameter('e'),init_qbt)
ansatz_opt2 = ansatz_opt3.compose(ansatz_opt2)

ansatz_opt = ansatz_opt.compose(ansatz_opt2)
meas_lst.extend(meas_lst2)
wt_list.extend(wt_list2)

In [21]:
#----------------7th row----------------------
init_qbt = 90
end_qbt = 102

ent = [(i,i+1) for i in range(init_qbt,end_qbt)]

ansatz_opt2 = TwoLocal(total_qubits, ['ry'], 'cx',  entanglement =  ent, reps=1, \
                      skip_final_rotation_layer=False, flatten=True, parameter_prefix='θ1',skip_unentangled_qubits=True)


meas_lst2 = [[88,90],[89,90],[90,91],[90,92],[91,92],[92,93],[91,94],[91,95],\
             [94,95],[95,96],[96,97],[95,97],[97,98],[97,99],[98,99],[99,100],[100,101]]  

ansatz_opt2 = zero_last_param(ansatz_opt2)

ansatz_opt2 = ansatz_opt2.assign_parameters(pval_103)

wt_list2 = [1 for i in range(len(meas_lst2))]

opf_lst = [[99,100]]
for pair in opf_lst:
    wt_list2[meas_lst2.index(pair)]=1.86
    

# [ansatz_opt,hamilt_single]=snip_ckt(90,ansatz_opt2,meas_lst2,True, wt_list2)

# triangles102 = {0:[[88,90],[89,90]]} 
triangles102 = {1:[[90,91],[90,92],[91,92]],2:[[92,93]],3:[[91,94],[91,95],\
                                                           [94,95]],4:[[95,96],[96,97],[95,97]],5:[[97,98],[97,99],[98,99]],6:[[99,100]],7:[[100,101]]}

lst_triangle_hamilt_102 = triangle_hamiltonian(triangles102,ansatz_opt,meas_lst2,wt_list2,90)

#---important tweak-------
ansatz_opt3 = QuantumCircuit(total_qubits)
ansatz_opt3.ry(Parameter('f'),init_qbt)
ansatz_opt2 = ansatz_opt3.compose(ansatz_opt2)

ansatz_opt = ansatz_opt.compose(ansatz_opt2)
meas_lst.extend(meas_lst2)
wt_list.extend(wt_list2)

In [22]:
#--------assign junction parameters manually, append missing observables-----------

# ansatz_opt = ansatz_opt.assign_parameters([5.02641207, -0.21759044, 11.65193109,  5.20588843,  4.47540369,  2.08801694])

junct_meas_lst = [[17,18],[17,19],[31,32],[30,32],[47,48],[47,49],[63,64],[63,65],[77,78],[76,78],[88,90],[89,90]]

#------add potentially missing observables at the junction----------
for pair in junct_meas_lst:
    if pair not in meas_lst:
        # print(pair)
        meas_lst.append(pair)
        wt_list.append(1)

unmain = [[89,99],[89,100],[58,90],[62,64],[60,88],[60,87],[63,84],[63,85],[66,82],[66,81],[69,79],[69,78],[42,70],\
          [42,71],[45,67],[45,68],[48,64],[48,65],[51,61],[51,62],[22,50],[22,49],[25,46],[25,47],[28,43],[28,44],\
            [31,40],[31,41],[15,23],[15,24],[9,29],[9,30],[6,32],[6,33],[33,36],[33,35]]
#------add potentially missing observables at the junction----------
for pair in unmain:
    if pair not in meas_lst:
        # print(pair)
        meas_lst.append(pair)
        wt_list.append(1)

#----------default ansatz hamiltonian--------------
hamilt_ansatz =  meas2hamilt(meas_lst, num_qbts=ansatz_opt.num_qubits,\
                             Pawli_X=True,Pawli_Y=True,Pawli_Z=True,wt_list = wt_list,int_wt3=1)



In [23]:
#----selected hamilt ansatz--------
# 18,32,48,64,78,90
meas_lst_sel = [[17,18],[17,19],[18,19],[30,32],[31,32],[32,33],\
            [6,32],[47,48],[48,49],[47,49],[48,64],[48,65],[62,64],\
                [63,64],[64,65],[76,78],[77,78],[78,79],[69,79],[69,78],\
                    [90,91],[58,90]]

wt_list_sel = extract_wt_lst(meas_lst_sel,meas_lst,wt_list)
hamilt_ansatz =  meas2hamilt(meas_lst_sel, num_qbts=ansatz_opt.num_qubits,\
                             Pawli_X=True,Pawli_Y=True,Pawli_Z=True,wt_list = wt_list_sel,int_wt3=1)

# ansatz_opt.draw(output = 'mpl')

print(wt_list_sel)


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [24]:
#---------define ODR ref. ancilla Ansatz--------------
ent = [(i,i+1) for i in range(total_qubits-1)]
anc_opt = TwoLocal(total_qubits, ['ry'], 'cx',  entanglement =  ent, reps=1, \
                      skip_final_rotation_layer=False, flatten=True, parameter_prefix='θ1',skip_unentangled_qubits=True)

param_sec_half = anc_opt.parameters[103:]
val_sec_half = []
for i in range(53):
    val_sec_half.extend([0,0])

#---------------------------------------------
param_first_half = anc_opt.parameters[0:103]
val_first_half = []
ct = 1
for i in range(53):
    val_first_half.extend([(-1)**ct * (pi/2),pi])
    ct += 1
#------merge the two pairs of lists-----------
param_first_half.extend(param_sec_half)
val_first_half.extend(val_sec_half)
param_val_dic = dict(zip(param_first_half,val_first_half))

anc_opt = anc_opt.assign_parameters(param_val_dic)

#------------default anc_hamilt--------------------
hamilt_anc =  meas2hamilt(meas_lst, num_qbts=anc_opt.num_qubits,\
                             Pawli_X=True,Pawli_Y=True,Pawli_Z=True,wt_list = [1 for i in range(len(meas_lst))],int_wt3=1)

In [159]:
#-------------obtain sub-circuit for classical simulation-------------

pivot_qbt = 21+1
#--------upper half, till pivot_qbt---------
#--------main ansatz-----------
LIST = list(range(21,31))
LIST.extend([15,9])
# ansatz_opt_new = truncate_circuit2(list(range(pivot_qbt)),ansatz_opt)
ansatz_opt_new = truncate_circuit2(LIST,ansatz_opt)
                                   
# meas_lst_new = trunc_meas_lst(list(range(pivot_qbt)),meas_lst)
meas_lst_new = trunc_meas_lst(LIST,meas_lst)

new_wt_lst = extract_wt_lst(meas_lst_new,meas_lst,wt_list)

hamilt_single = meas2hamilt(meas_lst_new, num_qbts=ansatz_opt_new.num_qubits,\
                            Pawli_X=True,Pawli_Y=True,Pawli_Z=True,wt_list = new_wt_lst,int_wt3=1)
#----now do for anc---------
# anc_opt_new = truncate_circuit2(list(range(pivot_qbt)),anc_opt)

#--------lower half, from pivot qbt---------
# [ansatz_opt_new,hamilt_single]=snip_ckt(pivot_qbt-1,ansatz_opt,meas_lst,True, wt_list)
# [anc_opt_new,hamilt_single]=snip_ckt(pivot_qbt-1,anc_opt,meas_lst,True, wt_list)




#------------set total qubits and Pauli observables------------
Total = total_qubits-1
Total = ansatz_opt.num_qubits - 1

[z_en, y_en, z_en] = [False, False, False]

TN = len(meas_lst)
print(meas_lst)

[[2, 3], [3, 4], [2, 4], [4, 5], [5, 6], [6, 7], [5, 7], [7, 8], [8, 9], [9, 10], [8, 10], [10, 11], [11, 12], [11, 13], [12, 13], [13, 14], [14, 15], [14, 16], [15, 16], [16, 17], [18, 19], [19, 20], [19, 21], [20, 21], [21, 22], [21, 23], [22, 23], [23, 24], [24, 25], [24, 26], [25, 26], [26, 27], [27, 28], [27, 29], [28, 29], [29, 30], [30, 31], [17, 18], [17, 19], [30, 32], [31, 32], [32, 33], [34, 35], [33, 35], [35, 36], [33, 36], [36, 37], [36, 38], [37, 38], [38, 39], [39, 40], [38, 40], [40, 41], [41, 42], [41, 43], [42, 43], [43, 44], [44, 45], [44, 46], [45, 46], [46, 47], [48, 49], [49, 50], [50, 51], [50, 52], [51, 52], [52, 53], [53, 54], [52, 54], [54, 55], [53, 56], [53, 57], [56, 57], [57, 58], [57, 59], [58, 59], [59, 60], [60, 61], [59, 61], [61, 62], [62, 63], [64, 65], [65, 66], [66, 67], [65, 67], [67, 68], [68, 69], [69, 70], [68, 70], [70, 71], [71, 72], [72, 73], [72, 74], [71, 74], [74, 75], [75, 76], [74, 76], [76, 77], [76, 78], [77, 78], [78, 79], [79, 80],

In [ ]:
# #--------assign final parameters manually----------
# ansatz_opt = ansatz_opt.assign_parameters([5.02641207, -0.21759044, 11.65193109,  5.20588843,  4.47540369,  2.08801694])
# # ansatz_opt_new.draw(output = 'mpl')
# ansatz_opt.parameters
# ansatz_opt_new.draw(output = 'mpl')
# meas_lst_new
# best real waala [ 5.43114599 -0.47206256 -0.84124728  4.69054087  4.64074419  1.60663808]

In [161]:


# ansatz_opt = ansatz_opt.compose(ansatz_opt2)
# meas_lst.extend(meas_lst2)
# wt_list.extend(wt_list2)

main = [[53,54],[54,55],[53,55],[55,56],[55,57],[56,57],[57,58],[57,59],[58,59],[59,60],\
      [60,61],[59,61],[61,62],[62,63],[62,64],[63,64],[64,65],[65,66],[66,67],[65,67],\
        [67,68],[68,69],[68,70],[70,71],[71,73],[72,73],[73,74],[71,74],[74,75],[75,76],\
            [74,76],[76,77],[76,78],[77,78],[78,79],[79,80],[79,81],[80,81],[81,22],[82,83],\
                [83,84],[82,84],[84,85],[85,86],[85,87],[86,87],[87,88],[88,89],[89,92],[88,92],\
                    [90,91],[91,92],[92,93],[91,93],[93,94],[94,95],[93,95],[95,96],[96,97],[95,97],\
                        [97,98],[97,99],[98,99],[99,100]]
unmain = [[89,99],[89,100],[60,88],[60,87],[63,84],[63,85],[66,82],[66,81],[69,79],[69,78],[42,70],\
          [42,71],[45,67],[45,68],[48,64],[48,65],[51,61],[51,62],[22,50],[22,49],[25,46],[25,47],[28,43],[28,44],\
            [31,40],[31,41]]

In [162]:
# #-----------Two Kagome Lattice Waala---------
# total_qubits = 19
# init_qbt = 0
# end_qbt = 18

# ent = [(i,i+1) for i in range(init_qbt,end_qbt)]
# # ent.extend([(18,0)])

# ansatz_opt = TwoLocal(total_qubits, ['ry'], 'cx',  entanglement =  ent, reps=1, \
#                       skip_final_rotation_layer=False, flatten=True, parameter_prefix='θ1',skip_unentangled_qubits=True)


# meas_lst = [[0,1],[1,2],[2,3],[2,4],[3,4],[4,5],[4,6],[5,6],[6,7],[6,8],[7,8],[8,9],[8,10],\
#             [9,10],[10,11],[10,12],[11,12],[12,13],[12,14],[13,14],[14,15],[14,16],[15,16],\
#                 [16,17],[17,18],[16,18],[1,11],[2,11],[0,17],[1,17]]

# wt_list = [1 for i in range(len(meas_lst))]

# # opf_lst = [[1,2]]
# # for pair in opf_lst:
# #     wt_list[meas_lst.index(pair)]=1.84

# hamilt_single = meas2hamilt(meas_lst, num_qbts=ansatz_opt.num_qubits,\
#                             Pawli_X=True,Pawli_Y=True,Pawli_Z=True,wt_list = wt_list,int_wt3=1)

# pval = [ 1.42564725, -0.21150297,  1.31315959,  3.15699189, -1.57156803,
#         3.03469103,  1.5867374 ,  0.00933507,  1.61305374, -0.01044298,
#         1.60411652,  3.03280637,  4.80979146,  2.98728448,  1.65013556,
#        -1.72729713,  3.30022435,  1.38679963, -0.21279638,  2.01924674,
#         5.12974905,  1.94708406,  1.98197096,  3.36566859,  3.35675407,
#         4.02545979,  0.91474037,  0.48332057,  3.70870419,  1.84010719,
#         2.01340901,  5.27439997,  5.22713156,  3.50091527,  6.17897391,
#        -0.2178978 ,  0.12069587,  3.20605053]

# # ansatz_opt = ansatz_opt.assign_parameters(pval)


# triangles = {-4:[[10,11],[10,12],[11,12]],-3:[[8,9],[9,10],[8,10]],-2:[[6,7],[7,8],[6,8]],-1:[[4,5],[4,6],[5,6]],\
#              0:[[2,3],[3,4],[2,4]], 2:[[1,2]], 1:[[0,1],[0,17],[1,17]]}

# lst_triangle_hamilt = triangle_hamiltonian(triangles,ansatz_opt,meas_lst,wt_list)

# Total = total_qubits-1
# Total = ansatz_opt.num_qubits - 1

# [z_en, y_en, z_en] = [False, False, False]

# TN = len(meas_lst)
# print(meas_lst)

In [163]:
print(wt_list)

[1, 1, 1, 1, 1, 1, 1, 1.8, 1, 1, 1, 1, 1, 1, 1, 1.8, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.84, 1, 1, 1, 1, 1, 1, 1, 1.84, 1, 1, 1, 1, 1, 1, 1, 1, 1.88, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.88, 1, 1, 1, 1, 1, 1.9, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.9, 1, 1, 1, 1, 1, 1.81, 1, 1, 1, 1, 1.81, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.85, 1, 1, 1, 1, 1, 1, 1, 1.85, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.86, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [164]:
#-------------Create the circuit circuit segments-------------
#-------------skip if not needed------------------------------
x_en = True
y_en = True
z_en = True

Nseg = 5

qbts_seg = [list(range(21+1)),list(range(21,41+1)),list(range(41,61+1)),list(range(61,81+1)),list(range(81,102))]

qbts_seg_anc = deepcopy(qbts_seg)

# [list(range(21+1)),list(range(21,41+1)),list(range(41,61+1)),list(range(61,81)),list(range(81,102))]

qbts_seg[1].extend([9,15])  # 21 to 41+1
qbts_seg[2].extend([22,28]) # 41 to 61+1
qbts_seg[3].extend([51,45]) # 61 to 81+1
qbts_seg[4].extend([60,66]) # 81 to end

ansatz_opt_lst = []
meas_lst_lst = []

hamilt_ansatz_lst = []
hamilt_anc_lst = []

anc_opt_lst = []


for i in range(Nseg):

    seg_ansatz_opt = truncate_circuit2(qbts_seg[i],ansatz_opt)
    ansatz_opt_lst.append(deepcopy(seg_ansatz_opt))             # must

    #------------update meas-lst and weight list ------------------
    meas_lst_new = trunc_meas_lst(qbts_seg[i],meas_lst)
    new_wt_lst = extract_wt_lst(meas_lst_new,meas_lst,wt_list)
    # print(meas_lst_new)
    # print(new_wt_lst)
    # print('------------------------')

    #-------
    seg_anc_opt = truncate_circuit2(qbts_seg[i],anc_opt)  
    anc_opt_lst.append(deepcopy(seg_anc_opt))                   # must

    seg_meas_lst = trunc_meas_lst(qbts_seg[i],meas_lst)
    seg_wt_lst = extract_wt_lst(meas_lst_new,meas_lst,wt_list)


    meas_lst_lst.append(deepcopy(meas_lst_new))

    #------now make hamiltonian---------
    hamilt_single = meas2hamilt(seg_meas_lst, num_qbts=seg_ansatz_opt.num_qubits,\
                                Pawli_X=x_en,Pawli_Y=y_en,Pawli_Z=z_en,wt_list = seg_wt_lst,int_wt3=1)
    hamilt_ansatz_lst.append(deepcopy(hamilt_single))

    hamilt_single = meas2hamilt(seg_meas_lst, num_qbts=seg_anc_opt.num_qubits,\
                                Pawli_X=x_en,Pawli_Y=y_en,Pawli_Z=z_en,wt_list = [1 for i in range(len(seg_meas_lst))],int_wt3=1)
    hamilt_anc_lst.append(deepcopy(hamilt_single))



In [116]:
estimator = EstimatorV2(backend1)   # backend-1 is aer

# job = estimator.run([(ansatz_opt,hamilt_single,pval)])
# job = estimator.run([(ansatz_opt,hamilt_single)])
job = estimator.run([(ansatz_opt,lst_triangle_hamilt)])

# job = estimator.run([(ansatz_opt_new,hamilt_single)])
# job = estimator.run([(anc_opt_new,hamilt_single)])

result = job.result()
pub_result = result[0]
print(pub_result.data.evs)

ansatz_opt_new.num_qubits

NameError: name 'EstimatorV2' is not defined

In [ ]:
# u = meas_lst.index([35,36])
# print(wt_list[u])
# len(hamilt_single)
# # len(meas_lst)
# len(wt_list)
# # hamilt_single
# total_qubits
# meas_lst
# ansatz_opt.draw(output = 'mpl')
# meas_lst

In [ ]:
#-----------exact solver------------
# hamilt_single = meas2hamilt(meas_lst,num_qbts=ansatz_opt.num_qubits,Pawli_X=True,Pawli_Y=True,Pawli_Z=True,int_wt3=-1)
hamilt_single
# ansatz_opt.draw(output = 'mpl')
from qiskit_algorithms.eigensolvers import NumPyEigensolver
# find the first three (k=3) eigenvalues
exact_solver = NumPyEigensolver(k=1)
exact_result = exact_solver.compute_eigenvalues(hamilt_single)
print(exact_result.eigenvalues)

[-29.14616811]


In [ ]:
# ansatz_opt_new.draw(output = 'mpl')
# ansatz_opt.parameters[3:5]
# for i in hamilt_single:
#     print(i)

In [ ]:
#-------apply circuit snipping function and generate hamiltonian----------

# ansatz_opt_1,meas_lst_1 = snip_ckt(35,ansatz_opt,meas_lst)
# print(meas_lst)
#                                                                                                                     # meas_lst_1.extend(trunc_meas_lst(list(range(35-35,82-35)),orig_meas_lst))

# hamilt_single_1 = meas2hamilt(meas_lst_1,num_qbts=ansatz_opt_1.num_qubits,Pawli_X=True,Pawli_Y=True,Pawli_Z=True)

# ansatz_opt_2,meas_lst_2 = snip_ckt(20,ansatz_opt,meas_lst)
# # print(meas_lst)
# hamilt_single_2 = meas2hamilt(meas_lst_2,num_qbts=ansatz_opt_2.num_qubits,Pawli_X=True,Pawli_Y=True,Pawli_Z=True)

# ansatz_opt_3,meas_lst_3 = snip_ckt(0,ansatz_opt,meas_lst)
# # print(meas_lst)
# hamilt_single_3 = meas2hamilt(meas_lst_3,num_qbts=ansatz_opt_3.num_qubits,Pawli_X=True,Pawli_Y=True,Pawli_Z=True)

#---------
# ansatz_opt,meas_lst = snip_ckt(70,ansatz_opt,meas_lst)
# hamilt_single = meas2hamilt(meas_lst,num_qbts=ansatz_opt.num_qubits,Pawli_X=True,Pawli_Y=True,Pawli_Z=True)

In [ ]:
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import EstimatorV2 
backend = simulator = AerSimulator(method='matrix_product_state')


# ansatz_opt = transpile(ansatz_opt, backend)  

AerSimulator('aer_simulator_matrix_product_state')


In [ ]:
from qiskit_ibm_runtime import EstimatorV2 

# pval =

# ansatz_opt = transpile(ansatz_opt, backend1)
estimator = EstimatorV2(backend)   # backend-1 is aer

# job = estimator.run([(ansatz_opt,hamilt_single,pval)])

# job = estimator.run([(ansatz_opt,hamilt_single)])
# job = estimator.run([(ansatz_opt,lst_triangle_hamilt_102)])

# job = estimator.run([(ansatz_opt_new,hamilt_single)])
job = estimator.run([(anc_opt_new,hamilt_single)])

#---------
# ind = 2
# job = estimator.run([(ansatz_opt_lst[ind],hamilt_ansatz_lst[ind])])
# # job = estimator.run([(anc_opt_lst[ind],hamilt_ansatz_lst[ind])])


result = job.result()

pub_result = result[0]

print(pub_result.data.evs)

-32.550732421875


In [ ]:

# data

# # print(ansatz_opt_1.num_qubits)
# # print(ansatz_opt_2.num_qubits)
# # print(ansatz_opt_3.num_qubits)
# # print(meas_lst_1)
# # ansatz_opt.num_parameters
# # hamilt_single[0]
# # meas_lst
# # ansatz_opt.draw(output = 'mpl')

In [ ]:
# # disent_ansatz_opt.draw(output = 'mpl')
# ansatz_opt.draw(output = 'mpl')
# new_ansatz.draw(output = 'mpl')
# print(data)
ansatz_opt.depth()

123

In [107]:
#-------------Create the circuit circuit segments-------------
#-------------skip if not needed------------------------------
x_en = True
y_en = True
z_en = True

Nseg = 5

qbts_seg = [list(range(21+1)),list(range(21,41+1)),list(range(41,61+1)),list(range(61,81+1)),list(range(81,102))]

qbts_seg_anc = deepcopy(qbts_seg)

# [list(range(21+1)),list(range(21,41+1)),list(range(41,61+1)),list(range(61,81)),list(range(81,102))]

qbts_seg[1].extend([9,15])  # 21 to 41+1
qbts_seg[2].extend([22,28]) # 41 to 61+1
qbts_seg[3].extend([51,45]) # 61 to 81+1
qbts_seg[4].extend([60,66]) # 81 to end

ansatz_opt_lst = []
meas_lst_lst = []

hamilt_ansatz_lst = []
hamilt_anc_lst = []

anc_opt_lst = []


for i in range(Nseg):

    seg_ansatz_opt = truncate_circuit2(qbts_seg[i],ansatz_opt)
    ansatz_opt_lst.append(deepcopy(seg_ansatz_opt))             # must

    #------------update meas-lst and weight list ------------------
    meas_lst_new = trunc_meas_lst(qbts_seg[i],meas_lst)
    new_wt_lst = extract_wt_lst(meas_lst_new,meas_lst,wt_list)

    #-------
    seg_anc_opt = truncate_circuit2(qbts_seg[i],anc_opt)  
    anc_opt_lst.append(deepcopy(seg_anc_opt))                   # must

    seg_meas_lst = trunc_meas_lst(qbts_seg[i],meas_lst)
    seg_wt_lst = extract_wt_lst(meas_lst_new,meas_lst,wt_list)


    meas_lst_lst.append(deepcopy(meas_lst_new))

    #------now make hamiltonian---------
    hamilt_single = meas2hamilt(seg_meas_lst, num_qbts=seg_ansatz_opt.num_qubits,\
                                Pawli_X=x_en,Pawli_Y=y_en,Pawli_Z=z_en,wt_list = seg_wt_lst,int_wt3=1)
    hamilt_ansatz_lst.append(deepcopy(hamilt_single))

    hamilt_single = meas2hamilt(seg_meas_lst, num_qbts=seg_anc_opt.num_qubits,\
                                Pawli_X=x_en,Pawli_Y=y_en,Pawli_Z=z_en,wt_list = [1 for i in range(len(seg_meas_lst))],int_wt3=1)
    hamilt_anc_lst.append(deepcopy(hamilt_single))



In [ ]:
# ind = 4
# anc_opt.draw(output = 'mpl')
# ansatz_opt_lst[ind].draw(output = 'mpl')
# # print(meas_lst_lst[ind])
len(hamilt_single)
# len(hamilt_single_anc)

# truncate_circuit2(qbts_seg[i],ansatz_opt)
# seg_meas_lst = trunc_meas_lst(qbts_seg[i],meas_lst)
# meas2hamilt(seg_meas_lst,seg_ansatz_opt.num_qubits,Pawli))

In [108]:
# len(hamilt_single_1)
# meas_lst

In [76]:
total_val = 0

# from qiskit_ibm_runtime import QiskitRuntimeService, Session, EstimatorV2

# ansatz_opt = transpile(ansatz_opt, backend)

# estimator = EstimatorV2(backend = backend)

with Session(backend=backend1) as session:
    estimator = EstimatorV2(backend1)
    # estimator = EstimatorV2(backend)
    N = len(meas_lst)
    # N = len(hamilt_single)

#     for i in range(len(hamilt_single)):
    # for triang_hamilt in range(len(lst_triangle_hamilt)):
#   
    for i in range(3*N):

    # for i in range(1):
       
        # job = estimator.run([(ansatz_opt,hamilt_single[i])])
        job = estimator.run([(ansatz_opt,hamilt_single[i])])
        # job = estimator.run([(ansatz_opt_2,hamilt_single_2)])
        # job = estimator.run([(ansatz_opt,hamilt_single[i],pval)])
        
        # job = estimator.run([(anc_opt_new,hamilt_single[i])])
        # job = estimator.run([(ansatz_opt_new,hamilt_single[i])])

        # job = estimator.run([(ansatz_opt,lst_triangle_hamilt)])
        
        result = job.result()
        pub_result = result[0]

        value1 = pub_result.data.evs
        # print(i)
        print(meas_lst[int(i/3)])
        # print(hamilt_single[i], value1)
        # print(i, value1)
        print(value1)
        # print(value1)
        total_val += value1

print(total_val)

[0, 1]
-0.00439453125
[0, 1]
0.0087890625
[0, 1]
-0.0283203125
[1, 2]
-0.9951171875
[1, 2]
-0.98486328125
[1, 2]
-0.9912109375
[2, 3]
-0.07421875
[2, 3]
0.01171875
[2, 3]
0.005859375
[2, 4]
0.06591796875
[2, 4]
-0.0244140625
[2, 4]
-0.0048828125
[3, 4]
-0.99853515625
[3, 4]
-0.998046875
[3, 4]
-0.9970703125
[4, 5]
-0.00146484375
[4, 5]
-0.001953125
[4, 5]
0.00830078125
[4, 6]
0.0166015625
[4, 6]
-0.02099609375
[4, 6]
0.015625
[5, 6]
-0.72412109375
[5, 6]
-0.66845703125
[5, 6]
-0.478515625
[6, 7]
-0.73095703125
[6, 7]
-0.65478515625
[6, 7]
-0.52099609375
[6, 8]
0.09130859375
[6, 8]
0.13720703125
[6, 8]
0.009765625
[7, 8]
-0.12939453125
[7, 8]
-0.16650390625
[7, 8]
-0.046875
[8, 9]
-0.8955078125
[8, 9]
-0.9208984375
[8, 9]
-0.94189453125
[8, 10]
-0.0751953125
[8, 10]
-0.02392578125
[8, 10]
-0.00390625
[9, 10]
0.05224609375
[9, 10]
0.0107421875
[9, 10]
-0.005859375
[10, 11]
-0.99560546875
[10, 11]
-0.998046875
[10, 11]
-0.9912109375
[10, 12]
-0.03271484375
[10, 12]
0.0205078125
[10, 12]
0

In [49]:
# index = 19
# print(qbts_seg[index])
# print(meas_lst_lst[index])
# hamilt_ = hamilt_single_lst[index]
# hamilt_[0]
# ansatz_opt_lst[index].draw(output = 'mpl',scale=0.3)
# len(hamilt_single)

# ansatz_opt.draw(output = 'mpl')
len(hamilt_single)

264

In [120]:
# a = service.backends(operational = True)#, simulator = False)
# backend=backends[0]
# print(backends)
# # a=a.properties()
# # a.gate_error(0)
# # b = backend.target['ecr'][(1,2)]
# b = backend.target['sx']
# print(b)
service.backends()

[<IBMBackend('ibm_brussels')>,
 <IBMBackend('ibm_strasbourg')>,
 <IBMBackend('test_eagle_eu-de')>,
 <IBMBackend('ibm_aachen')>]

[<IBMBackend('ibm_brisbane')>, <IBMBackend('ibm_fez')>, <IBMBackend('ibm_kyiv')>, <IBMBackend('ibm_sherbrooke')>, <IBMBackend('ibm_torino')>, <IBMBackend('ibm_marrakesh')>, <IBMBackend('ibm_kingston')>]


In [50]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager


In [293]:
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)

isa_psi_lst = []
isa_psi_anc_lst = []
#---------April 4, 2025-----------------
isa_hamilt_ansatz_lst = []
isa_hamilt_anc_lst = []

opt_level = 1
for i in range(Nseg):

    pm = generate_preset_pass_manager(backend=backend, optimization_level=opt_level)
    isa_psi = pm.run(ansatz_opt_lst[i])
    isa_psi_lst.append(pm.run(isa_psi))

    isa_hamilt_ansatz_lst.append(hamilt_ansatz_lst[i].apply_layout(isa_psi.layout))
    
    init_layout = isa_psi.layout.final_index_layout()
    # print(init_layout)

    pm = generate_preset_pass_manager(backend=backend, optimization_level=opt_level,initial_layout=init_layout)
    isa_psi_anc_lst.append(pm.run(anc_opt_lst[i]))
    isa_hamilt_anc_lst.append(hamilt_anc_lst[i].apply_layout(isa_psi.layout))
    # isa_hamilt_anc_lst.append(hamilt_anc_lst[i].apply_layout(isa_psi_anc_lst[i].layout))
    # print(isa_psi_anc_lst[i].layout.final_index_layout())

In [227]:
# isa_psi.draw(output = 'mpl')
print(isa_psi_anc_lst[1].layout.final_index_layout())
print(isa_psi_lst[1].layout.final_index_layout())

[16, 23, 22, 21, 20, 19, 15, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 18, 31, 32, 33, 37, 52, 51, 50, 56, 69, 68, 67, 66, 65, 55, 46, 47, 48, 36, 29, 28, 27, 17]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132]


In [ ]:
#------ancilla if needed---------

# init_layout = isa_psi.layout.final_index_layout()
# pm = generate_preset_pass_manager(backend=backend, optimization_level=gloab_opt_lev,initial_layout=init_layout)
# isa_anc_psi = pm.run(anc_opt)
# isa_hamilt_anc = hamilt_single_anc.apply_layout(isa_anc_psi.layout)

# a = isa_psi.layout
# # print(a)
# a = a.final_index_layout()
# print(a)

# print(isa_anc_psi.depth())
# print(isa_anc_psi.num_nonlocal_gates())

In [294]:
for i in range(Nseg):
    ee = isa_psi_lst[i].layout.final_index_layout()
    # ee = isa_psi_lst[i].layout
    print(ee)

    print(isa_psi_anc_lst[i].num_nonlocal_gates())
    print(isa_psi_anc_lst[i].depth())

    print(isa_psi_lst[i].num_nonlocal_gates())
    print(isa_psi_lst[i].depth())
    print('------------------')

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155]
22
91
22
93
------------------
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76,

In [35]:
pval1 = [5.02641207, -0.21759044, 11.65193109,  5.20588843,  4.47540369,  2.08801694] # sim vqe optimal

pval2 = [ 4.5446756 ,  0.07960752,  2.04610456, -0.08805339,  1.7419572 , 2.31264188] # real vqe optimal

pval3 = [4.36759143, 0.40973846, 0.32851966, 0.1279157,  1.67734968, 2.02703544] # real vqe int. -83.14384705265131 ?

pval4 = [4.36759143, 0.40973846, 2.08863394, 1.69871203, 1.67734968, 2.02703544] # real vqe int. -78sh? 



In [295]:
#----------Run estimator in Job mode---------
from qiskit_ibm_runtime import EstimatorV2 as Estimator

estimator = Estimator(mode = backend)

options = estimator.options

estimator.options.resilience.measure_mitigation = True
# estimator.options.resilience.measure_noise_learning.num_randomizations = 8
# estimator.options.resilience.measure_noise_learning.shots_per_randomization = 32

estimator.options.dynamical_decoupling.enable = True

estimator.options.dynamical_decoupling.sequence_type = "XpXm"
estimator.options.dynamical_decoupling.skip_reset_qubits = True

estimator.options.twirling.enable_gates = True
estimator.options.twirling.num_randomizations = "auto"

estimator.options.resilience.zne_mitigation = True
estimator.options.resilience.zne.noise_factors = (1,1.01,1.02, 1.03, 1.04, 1.05)# , 1.06, 1.07) # (1,2,3) 
# estimator.options.resilience.zne.extrapolator = ("exponential","linear")

                                # estimator.options.resilience.zne_mitigation = True
                                # estimator.options.resilience.zne.amplifier = "pea"

estimator.options.default_shots = 512

# job = estimator.run([(isa_psi_lst[i], isa_hamilt_ansatz_lst[i]) for i in range(Nseg)])
# print(job.job_id())
# job = estimator.run([(isa_psi_lst[i], isa_hamilt_ansatz_lst[i]) for i in range(Nseg)])
# print(job.job_id())

job = estimator.run([(isa_psi_lst[i], isa_hamilt_ansatz_lst[i]) for i in range(Nseg)])
print(job.job_id())
job = estimator.run([(isa_psi_anc_lst[i], isa_hamilt_anc_lst[i]) for i in range(Nseg)])
print(job.job_id())


# job = estimator.run([(isa_psi, isa_hamilt),(isa_anc_psi, isa_hamilt_anc)])
# print(job.job_id())

cvookonlv4ac739vvpbg
cvooko3n8pnc73c86c2g


In [51]:
import statistics
# anc cvpcfhp25hpc73du0bt0
# ansatz pval4 cvpcfivlv4ac73a35vjg

# job = service.job('cvpbenv4kg4s73e66log')
job = service.job('cvpcfivlv4ac73a35vjg')

# cvocqmp25hpc73dp5850 # anc
# cvocqnvlv4ac739uaqdg # ansatz

# 
job_result = job.result()
# print(job_result) # job_result.dat
sum_evs = 0
# job_result['PubResult']
# print(job_result)
for idx, pub_result in enumerate(job_result):
    print(f"{idx}: {pub_result.data.evs}")
    # sum_evs += pub_result.data.evs

    #--------use for ZNE-------------
    # print(f"pub {idx}: {pub_result.data.evs_noise_factors}")
    # print("pub {idx}: {pub_result.data.evs_noise_factors}")

    # print(f"Expectation values for pub {idx}: {pub_result.data.evs}")
    # sum_evs += statistics.mean(pub_result.data.evs_noise_factors)
    # print(f"std values for pub {idx}: {pub_result.data.stds}")
    # print(pub_result.data)

print(sum_evs)

0: -78.30498639260213
0


In [ ]:
#----------Run estimator in Batch---------
from qiskit_ibm_runtime import EstimatorV2 as Estimator, Batch


# estimator = Estimator(backend = backend)
with Batch(backend=backend) as batch:
    estimator = Estimator(batch)

    options = estimator.options

    estimator.options.resilience.measure_mitigation = True
    # estimator.options.resilience.measure_noise_learning.num_randomizations = 8
    # estimator.options.resilience.measure_noise_learning.shots_per_randomization = 64

    estimator.options.dynamical_decoupling.enable = True
    estimator.options.dynamical_decoupling.sequence_type = "XpXm"
    estimator.options.dynamical_decoupling.skip_reset_qubits = True


    estimator.options.twirling.enable_gates = True
    estimator.options.twirling.num_randomizations = "auto"

    estimator.options.resilience.zne_mitigation = True
    estimator.options.resilience.zne.noise_factors = (1,2,3)
    estimator.options.resilience.zne.extrapolator = ("exponential","linear")

                                # estimator.options.resilience.zne_mitigation = True
                                # estimator.options.resilience.zne.amplifier = "pea"

    estimator.options.default_shots = 512

    job = estimator.run([(isa_psi, isa_hamilt),(isa_anc_psi, isa_hamilt_anc)])
    print(job.job_id())
    job = estimator.run([(isa_psi, isa_hamilt),(isa_anc_psi, isa_hamilt_anc)])
    print(job.job_id())
    job = estimator.run([(isa_psi, isa_hamilt),(isa_anc_psi, isa_hamilt_anc)])
    print(job.job_id())


    # job = estimator.run([(isa_anc_psi, isa_hamilt_anc)])
    # print(job.job_id())

    # # len(isa_psi_lst)
    # job = estimator.run([(isa_psi_lst[i, isa_hamilt_lst[i]) for i in range(Nseg)])
    # print(job.job_id())
    
    

In [191]:
# job = service.job('ct8vsd43lae6ijlv4mi0')
# job.status()
print(job)

<RuntimeJobV2('cvo2tqh25hpc73dngk8g', 'estimator')>


In [26]:
import statistics

# job1 = service.job('ct5lvba87o5f8kriqms0')
job1 = service.job('crhhqoh9kq9r0q2lnsmg')
job_list = [job1]

#-----or uncomment below for the session jobs------------
# job_list = service.jobs(100,0,session_id = 'c833293f-44a6-43d8-8061-29971e4ee4c5')

# print(job1.status())
# print(job2.status())
values = []
job_ct = 0

print('started')

lst_param_val = []
lst_obs_val = []

for job in job_list:
    job_result = job.result()
    # print(job_result)
    sum_evs = 0
    d = job.inputs
    # print('parameter values', d['pubs'][0][2])
    lst_param_val.append(d['pubs'][0][2])

    for idx, pub_result in enumerate(job_result):
        print(f"Expectation values for pub {idx}: {pub_result.data.evs}")
        value = statistics.mean(pub_result.data.evs_noise_factors)
        print(pub_result.data.evs_noise_factors)
        # value = pub_result.data.evs_noise_factors                   
        print(value)
        # lst_obs_val.append(value)
        # values.append(pub_result.data.evs)
        # values.append(statistics.mean(pub_result.data.evs_noise_factors))
    # print('-----------------------')
    job_ct += 1

started
Expectation values for pub 0: 105.85120925036118
[8.40146744 9.36603456 8.54024249 8.19314466 8.8642242 ]
8.673022670905382
Expectation values for pub 1: -200.58008743583036
[7.56377905 7.34203442 7.43898843 7.67234674 7.69713783]
7.542857294217246
Expectation values for pub 2: -166.92595575104212
[7.99975636 7.47239387 7.92350447 8.35378645 7.79829979]
7.909548189456249
Expectation values for pub 3: -653.7169365330526
[6.12156047 6.06641036 6.64938168 6.75675851 6.08497092]
6.3358163877938605
Expectation values for pub 4: -405.3229141917385
[5.39645225 5.08638941 6.13631718 5.25344224 5.62762312]
5.500044841172355


In [ ]:
# from qiskit_aer import AerSimulator
# from qiskit_ibm_runtime import EstimatorV2 
# backend = AerSimulator()

from qiskit_ibm_runtime import QiskitRuntimeService
service = QiskitRuntimeService(channel="ibm_cloud", token="**********", instance="Qiskit Runtime-SimOnly")

bends = service.backends()
backend = bends[2]
print(backend)

<IBMBackend('simulator_mps')>


In [ ]:
# total_val = 0
# N = len(meas_lst)
#with Session() as session:
# ------------------------------------

estimator = EstimatorV2(backend = backend)

ct = 0
for pvl in lst_param_val:
    job = estimator.run([(ansatz_opt,hamilt_single,pvl)])
    result = job.result()
    pub_result = result[0]
    value1 = pub_result.data.evs
    # print(i)
    print('denoised_val',value1)
    print('actual_val',lst_obs_val[ct])
    ct += 1


In [194]:
# values.reverse()
# plt.plot(list(range(job_ct)),values, marker='o', label='100-qubit GHZ state')
# # plt.xlabel('Distance between qubits $i$')
# # plt.ylabel(r'$\langle Z_i Z_0 \rangle / \langle Z_1 Z_0 \rangle $')
# # plt.legend()
# # plt.show()

In [ ]:
print(list(values))

d = job.inputs
ckt = d['pubs'][0][0]
# ckt = QuantumCircuit(12)
a= ckt.qregs
print(a)

# service.

In [ ]:
import statistics
# job = service.job('cwzgj6g5v39g008j7djg')
# job = service.job('csv2no81um50tav4vn50')
job = service.job('csvccadia22tu9r3mcv0')
# 
# job_result = [job.result()[0]]
job_result = job.result()
job_result = job.result()
# print(job_result) # job_result.dat

# job_result['PubResult']
# print(job_result)
sum_evs = 0
for idx, pub_result in enumerate(job_result):
    print(f"Expectation values for pub {idx}: {pub_result.data.evs}")
    print(f"Expectation values for pub {idx}: {pub_result.data.evs_noise_factors}")
    # sum_evs += statistics.mean(pub_result.data.evs_noise_factors)
    print(statistics.mean(pub_result.data.evs_noise_factors))
    # print(sum(pub_result.data.evs_noise_factors)/5)
    # print(f"stds values for pub {idx}: {pub_result.data.stds}")
    # print(pub_result.data)

print(sum_evs)

In [ ]:
# job.inputs
# ckt.draw()
job_result = job.result()[0]
job_result.data.evs

In [23]:
# d = service.jobs()
# print(d)

# print(hamilt_)

# fuck

In [ ]:
chacha_ct = 0

for meas_lst1 in meas_lst_lst:
    for qoubits in meas_lst1:
        if len(qoubits) == 2:
            chacha_ct += 1
print(chacha_ct)

print(meas_lst_lst)
print(meas_lst)
-6.05/(33/126)

In [194]:
# ansatz_opt_lst[4].draw(output = 'mpl')
# isa_psi_anc_lst[0].draw(output = 'mpl')

In [ ]:
#-------------Run Samplerin Batch---------
from qiskit_ibm_runtime import SamplerV2, Batch, Session

with Batch(backend=backend) as batch:
    sampler = SamplerV2(session = Batch)
    job = sampler.run(isa_psi_lst, shots=200)
    print(job.job_id())

    job = sampler.run(isa_psi_anc_lst, shots=200)
    print(job.job_id())


In [26]:
# ---------------Custom VQE------------------------
import statistics
from random import randint
from qiskit_algorithms import MinimumEigensolver, VQEResult

# Define a custome VQE class to orchestra the ansatz, classical optimizers, 
# initial point, callback, and final result
class CustomVQE(MinimumEigensolver):
    
    def __init__(self, estimator, circuit, optimizer, callback=None):
    # def __init__(self, estimator, circuit, anc_circuit, optimizer, callback=None):

        self._estimator = estimator
        # self._circuit = circuit
        self._circuit = circuit
        # self._anc_circuit = anc_circuit
        self._optimizer = optimizer
        self._callback = callback
        
    def compute_minimum_eigenvalue(self, operators_lst, aux_operators=None):
    # def compute_minimum_eigenvalue(self, operators_lst, operators_lst2, aux_operators=None):

                
        # Define objective function to classically minimize over
        def objective(x):
            # Execute job with estimator primitive
            # job = self._estimator.run(self._circuit, operators_lst, [x]*len(operators_lst))
            
            # self._estimator.options.default_shots = 200
            
        #-------- double down code with ansatz
      
            # job = self._estimator.run([(self._circuit, operators_lst, x) ,(self._anc_circuit,operators_lst2)])

            # est_result = [job.result()[0]]  # for ref with anc

            # value_orig = 0
            # for idx, pub_result in enumerate(est_result):
            #     value_orig =  pub_result.data.evs
            #     #----------ZNE---------------
            # #     value_orig =  statistics.median(pub_result.data.evs_noise_factors)
                #---------------------------

        #     est_result = [job.result()[1]]
        #     value_anc = 0
        #     for idx, pub_result in enumerate(est_result):
        #         value_anc =  pub_result.data.evs
        #         #----------ZNE---------------
        #     #     value_scale =  statistics.median(pub_result.data.evs_noise_factors)
        #         #----------------------------

        #    #--------Apply ODR--------------- 
        #     value = (value_orig/value_anc)*(-150.0)



           #--------original code---------------
            job = self._estimator.run([(self._circuit, operators_lst, x)])
            est_result = job.result()
            
            for idx, pub_result in enumerate(est_result):
                value =  pub_result.data.evs

                #---------zne------------
                # value =  statistics.median(pub_result.data.evs_noise_factors)
                # value =  max(pub_result.data.evs_noise_factors)
                
            #--------end original code---------------
            # value = sum(est_result.values)
            print(value)
            print(x)
            # Save result information using callback function
            if self._callback is not None:
                # self._callback([value,[x]])
                self._callback(value)
                
            return value
 
        
        x0 = np.pi * np.random.rand(len(ansatz_opt.parameters))  # uncomment to set initial point to random parm. values
        # x0 = [ 1.51133236, -1.68235996, -1.03080574,  4.68031235,  4.1060408,   1.47716664]

        # x0.extend(-pi/2 for i in range(ansatz_opt.num_parameters-len(x0)))
        # x0 = []
        # pval = [pi/2,pi,-pi/2,pi]
        # x0 = [pi/2,pi,-pi/2,pi,pi/2,pi,-pi/2,pi,pi/2,pi,-pi/2,pi,pi/2,pi,-pi/2,pi,pi/2,pi,-pi/2]
        # x0.extend([0 for i in range(19)])
        # x0.append(pi)
                
       #  # --------------best waala-----------
        print(x0)
        
        res = self._optimizer.minimize(objective, x0=x0)
        # Populate VQE result
        result = VQEResult()
        result.cost_function_evals = res.nfev
        result.eigenvalue = res.fun
        result.optimal_parameters = res.x
        return result

In [27]:

# from qiskit.primitives import Estimator
from qiskit_algorithms.optimizers import SPSA, GradientDescent,NFT,COBYLA
from qiskit_ibm_runtime import EstimatorV2 
from qiskit_algorithms.minimum_eigensolvers import VQE
# from qiskit.primitives import Estimator
# pip install nlopt
# optimizer = SPSA(maxiter=1500)
# optimizer = CRS(max_evals=300)
intermediate_info = []
optimizer = NFT(maxiter =100)
# optimizer = COBYLA(maxiter = 700)
def callback(value):
    intermediate_info.append(value)

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService, Session, EstimatorV2
# service = QiskitRuntimeService()
from qiskit_aer import AerSimulator
backend1 = AerSimulator()

# from qiskit_ibm_runtime import QiskitRuntimeService
# service = QiskitRuntimeService(channel="ibm_cloud", token="**********", instance="Qiskit Runtime-SimOnly")
# bends = service.backends()
# backend = bends[7]

# # ansatz_opt = transpile(ansatz_opt, backend)
# print(backend)

In [79]:
intermediate_info = []
with Session( backend=backend1) as session:
#with Session() as session:
    for i in range(1):
        estimator = EstimatorV2(session)
        # estimator = Estimator()
        estimator.options.default_shots = 100 
        custom_vqe = CustomVQE(estimator, ansatz_opt, optimizer, callback=callback)
        # custom_vqe = CustomVQE(estimator, ansatz_opt_new, optimizer, callback=callback)
        result = custom_vqe.compute_minimum_eigenvalue(hamilt_single)
    print(result)
# session.close()

NameError: name 'Session' is not defined

In [502]:
# from qiskit_ibm_runtime import EstimatorV2, Batch
ansatz_opt_new.num_parameters

2

In [28]:
# print(service.backends())
# backend = service.backend("ibm_brussels",use_fractional_gates=True)
# backend = service.backend("ibm_strasbourg")
# backend = service.backend("ibm_aachen")
# backend = service.backend("test_eagle_eu-de",use_fractional_gates=True)

# backend = service.backend("ibm_osaka")
# backend = service.backend("ibm_brisbane",use_fractional_gates=True)
# backend = service.backend('ibm_sherbrooke')#,use_fractional_gates=True)
# backend = service.backend("ibm_kyiv",use_fractional_gates=True)

# backend = service.backend("ibm_fez",use_fractional_gates=False)
# backend = service.backend('ibm_torino',use_fractional_gates=False)
# backend = service.backend("ibm_marrakesh",use_fractional_gates=False)

backend = service.backend("ibm_kingston",use_fractional_gates=True)

print(backend)

<IBMBackend('ibm_kingston')>


In [ ]:
#-----------Get ready for real VQE------------------

In [62]:
#-------single ansatz_opt and single anc----------

In [31]:
opt_level = 2
# init = list(range(103))
pm = generate_preset_pass_manager(backend=backend, optimization_level=opt_level)#,initial_layout=init)
isa_psi = pm.run(ansatz_opt)
isa_hamilt_ansatz = hamilt_ansatz.apply_layout(isa_psi.layout)

print(isa_psi.depth())
print(isa_psi.num_nonlocal_gates())

#------ancilla if needed---------

init_layout = isa_psi.layout.final_index_layout()
pm = generate_preset_pass_manager(backend=backend, optimization_level=opt_level,initial_layout=init_layout)
isa_anc_psi = pm.run(anc_opt)
isa_hamilt_anc = hamilt_anc.apply_layout(isa_anc_psi.layout)

a = isa_psi.layout

a = a.final_index_layout()
print(a)

print(isa_anc_psi.depth())
print(isa_anc_psi.num_nonlocal_gates())

420
102
[38, 29, 28, 27, 17, 7, 8, 9, 10, 11, 18, 31, 32, 33, 39, 53, 52, 51, 58, 71, 70, 69, 68, 67, 57, 47, 46, 45, 37, 25, 24, 23, 22, 21, 36, 41, 42, 43, 56, 63, 62, 61, 76, 81, 82, 83, 96, 103, 102, 101, 116, 121, 122, 123, 124, 125, 117, 105, 106, 107, 97, 87, 88, 89, 90, 91, 98, 111, 110, 109, 118, 129, 128, 127, 137, 147, 148, 149, 150, 151, 152, 153, 154, 155, 139, 135, 134, 133, 119, 113, 114, 115, 99, 95, 94, 93, 79, 73, 74, 75, 59, 55, 54]
334
102


In [ ]:
'rRun Real VQE' 
from qiskit_ibm_runtime import EstimatorV2 as Estimator, Batch

intermediate_info = []

# with Session(
# 
# 
# 
# 
# 
# 
#  = service, backend=backend, max_time = '2h') as session:

with Batch(backend=backend) as batch:
    # estimator = Estimator()
    estimator = Estimator(batch)

    # estimator = EstimatorV2(session = session)
   
    estimator.options.resilience.measure_mitigation = True
    # estimator.options.resilience.measure_noise_learning.num_randomizations = 8
    # estimator.options.resilience.measure_noise_learning.shots_per_randomization = 64
    estimator.options.dynamical_decoupling.enable = True

    estimator.options.dynamical_decoupling.sequence_type = "XpXm"

    estimator.options.twirling.enable_gates = True
    estimator.options.twirling.num_randomizations = "auto"

    # estimator.options.resilience.zne_mitigation = True
    # estimator.options.resilience.zne.noise_factors = (1, 1.01, 1.02, 1.03, 1.04)
    # estimator.options.resilience.zne.extrapolator = "linear"

    # # estimator.options.resilience.zne_mitigation = True
    # # estimator.options.resilience.zne.amplifier = "pea"

    estimator.options.default_shots = 256
                                                                                            #estimator = Estimator()
    # estimator.options.default_shots = 100

    #--------bare ansatz (no anc)------------
    custom_vqe = CustomVQE(estimator, isa_psi, optimizer, callback=callback)
    result = custom_vqe.compute_minimum_eigenvalue(isa_hamilt_ansatz)

    #---------with anc ODR-------------
    # custom_vqe = CustomVQE(estimator, isa_psi, isa_anc_psi, optimizer, callback=callback)
    # result = custom_vqe.compute_minimum_eigenvalue(isa_hamilt_ansatz,isa_hamilt_anc)

print(result)
# session.close()
batch.close()

[0.99964248 0.84213778 3.13832047 0.9126026  0.02165113 2.05786232]
-3.9562536708620795
[0.99964248 0.84213778 3.13832047 0.9126026  0.02165113 2.05786232]


In [ ]:
gloab_opt_lev = 1
# pm = generate_preset_pass_manager(backend=backend, optimization_level=gloab_opt_lev,scheduling_method="alap")#,initial_layout=init)
# isa_psi_1 = pm.run(ansatz_opt_1)
# isa_hamilt_1 = hamilt_single_1.apply_layout(isa_psi_1.layout)

# pm = generate_preset_pass_manager(backend=backend, optimization_level=gloab_opt_lev,scheduling_method="alap")#,initial_layout=init)
# isa_psi_2 = pm.run(ansatz_opt_2)
# isa_hamilt_2 = hamilt_single_2.apply_layout(isa_psi_2.layout)

# pm = generate_preset_pass_manager(backend=backend, optimization_level=gloab_opt_lev,scheduling_method="alap")#,initial_layout=init)
# isa_psi_3 = pm.run(ansatz_opt_3)
# isa_hamilt_3 = hamilt_single_3.apply_layout(isa_psi_3.layout)

# print(isa_psi_1.depth())
# print(isa_psi_1.num_nonlocal_gates())
# print(isa_psi_2.depth())
# print(isa_psi_2.num_nonlocal_gates())
# print(isa_psi_3.depth())
# print(isa_psi_3.num_nonlocal_gates())

pm = generate_preset_pass_manager(backend=backend, optimization_level=gloab_opt_lev,scheduling_method="alap")#,initial_layout=init)
isa_psi = pm.run(ansatz_opt)
isa_hamilt = hamilt_single.apply_layout(isa_psi.layout)

print(isa_psi.depth())
print(isa_psi.num_nonlocal_gates())

In [ ]:
pip install -U qiskit-ibm-runtime

In [48]:
pval1 = [5.02641207, -0.21759044, 11.65193109,  5.20588843,  4.47540369,  2.08801694] # sim vqe optimal

pval2 = [ 4.5446756 ,  0.07960752,  2.04610456, -0.08805339,  1.7419572 , 2.31264188] # real vqe optimal

pval3 = [4.36759143, 0.40973846, 0.32851966, 0.1279157,  1.67734968, 2.02703544] # real vqe int. -83.14384705265131 ?

pval4 = [4.36759143, 0.40973846, 2.08863394, 1.69871203, 1.67734968, 2.02703544] # real vqe int. -78sh? 

pval = [1.10454119, 0.53007528, 1.09413285, 0.42678031, 1.79488283, 2.21185581]

In [49]:
'rRun Real VQE' 
from qiskit_ibm_runtime import EstimatorV2 as Estimator, Batch

intermediate_info = []

# with Session(service = service, backend=backend, max_time = '2h') as session:

with Batch(backend=backend) as batch:
    # estimator = Estimator()
    estimator = Estimator(batch)

    # estimator = EstimatorV2(session = session)
   
    estimator.options.resilience.measure_mitigation = True
    # estimator.options.resilience.measure_noise_learning.num_randomizations = 8
    # estimator.options.resilience.measure_noise_learning.shots_per_randomization = 64
    estimator.options.dynamical_decoupling.enable = True

    estimator.options.dynamical_decoupling.sequence_type = "XpXm"

    estimator.options.twirling.enable_gates = True
    estimator.options.twirling.num_randomizations = "auto"

    # estimator.options.resilience.zne_mitigation = True
    # estimator.options.resilience.zne.noise_factors = (1, 1.01, 1.02, 1.03, 1.04)
    # estimator.options.resilience.zne.extrapolator = "linear"

    # # estimator.options.resilience.zne_mitigation = True
    # # estimator.options.resilience.zne.amplifier = "pea"

    estimator.options.default_shots = 512
                                                                                            #estimator = Estimator()
    # estimator.options.default_shots = 100

    job = estimator.run([(isa_anc_psi, isa_hamilt_anc)])
    print('anc',job.job_id())
    # job = estimator.run([(isa_psi, isa_hamilt_ansatz,pval1)])
    # print('ansatz pval1',job.job_id())
    # job = estimator.run([(isa_psi, isa_hamilt_ansatz,pval2)])
    # print('ansatz pval2',job.job_id())
    # job = estimator.run([(isa_psi, isa_hamilt_ansatz,pval3)])
    # print('ansatz pval3',job.job_id())
    job = estimator.run([(isa_psi, isa_hamilt_ansatz,pval)])
    print('ansatz pval4',job.job_id())

# session.close()
batch.close()

anc cvpcfhp25hpc73du0bt0
ansatz pval4 cvpcfivlv4ac73a35vjg
